In [3]:
%load_ext autoreload
%autoreload 2
from ds_utils.ds_helper import *
from ds_utils.ds_plotting import *
# from glob import glob
# import re
# import datetime
# import duckdb
# import pathlib
# from ds_utils.ds_preamble import *  # # self-defined moduleb
# from ds_utils.ds_helper import *  # self-defined module
# from ds_utils.ds_plotting import *  # # self-defined module

tips = sns.load_dataset('tips')
tips_tiny = tips.sample(20)

df = tips
np.random.seed(123)
tips['decile'] = np.random.randint(1,4, size=len(tips))

# import importlib
# importlib.reload(ds_preamble)
import math
from math import ceil, floor
from collections import Counter 
from functools import cache
# itertools: combinations, product
import itertools


import scipy as sp
from math import comb

sys.setrecursionlimit(100000)
import statsmodels.api as sm
from itertools import combinations


calling ds_helper...
calling self ds_preamble..


# todo
- how to determine if the numbers repeat itself?
- implement the distribution given support and probability distribution
	- pmf, cdf, sf, isf

# classic algorithms

## implement hasing (hash table)

In [48]:
# M1: constant size implementation
# class HashMap:
#     def __init__(self, n=100):
#         # just a list of n lists
#         self.maps = [[] for i in range(n)]
#     def find_map(self, k):
#         # find the map that value k maps to 
#         index = hash(k)% len(self.maps)
#         return self.maps[index] # return a list
#     def add(self, k, v):
#         # add a k,v pair to maps
#         m = self.find_map(k)
#         m.append((k,v))
#     def get(self, k):
#         m = self.find_map(k)
#         for key, val in m:
#             if key == k:
#                 return val
#         raise KeyError

class HashMap:
    def __init__(self, n=2):
        # just a list of n lists
        self.maps = [[] for i in range(n)]
        self.n_elements = 0
    def find_index(self, k):
        return hash(k)% len(self.maps)
    def find_map(self, k):
        # find the map that value k maps to 
        return self.maps[self.find_index(k)] # return a list
    def add(self, k, v):
        # add a k,v pair to maps
        if self.n_elements == len(self.maps):
            self.resize()
        m = self.find_map(k)
        m.append((k,v))
        self.n_elements += 1
    def get(self, k):
        m = self.find_map(k)
        for key, val in m:
            if key == k:
                return val
        raise KeyError
    def resize(self):
        print(f"resizing to {self.n_elements*2}...")
        new_size = self.n_elements*2
        new_maps = [[] for i in range(new_size)]
        for map in self.maps:
            for k, v in map:
                new_maps[hash(k)%new_size].append((k,v))
        self.maps = new_maps

map = HashMap()
map.add(1,2)
map.add(2,3)
map.maps
map.add(3,4)
map.add(4,4)
map.add(5,5)

map.maps

# map.add(1,2)
# map.add(2,3)
# map.add(3,4)
# map.get(3)
# map.add(101,2)
# map.maps
# map.get(101)

[[(2, 3)], [(1, 2)]]

resizing to 4...
resizing to 8...


[[], [(1, 2)], [(2, 3)], [(3, 4)], [(4, 4)], [(5, 5)], [], []]

## insertion sort
- idea: start from looking at `a[:1]`. It's already sorted. Then. look at `a[:2]`. We need to put `a[1]` into the correct position relative to `a[:2]`. Basically, we need to insert `a[i]` into the correct position of `a[:i+1]`, where `a[:i]` is already sorted.

In [16]:
def insertion_sort(a):
    for i in range(1, len(a)):
        # find correct insertion location
        j = i - 1
        candidate = a[i]
        while j>=0 and a[j] > candidate:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = candidate 

# a=[2,4,5,7,1,2,3,6]
# a=[1,3,7,2]
a = [2,1]
insertion_sort(a)
print(a)


# insertion_sort()



1
[1, 2]


## bubble sort

In [21]:
def bubble_sort(a):
    n = len(a)
    for i in range(n):
        for j in range(n-1-i):
            if a[j] > a[j+1]: # swap
                a[j], a[j+1] = a[j+1], a[j]
                
a=[2,4,5,7,1,2,3,6]
a=[1,3,7,2]
# a = [2,1]
bubble_sort(a)
print(a)
                
            


[1, 2, 3, 7]


## mergesort

In [33]:
def merge(s1,s2, s=None):
    # if s is None, then create a new array to store the result
    m, n = len(s1), len(s2)
    if s is None:
        s = [0] * (m+n)
    i=j =0
    
    # -------------------------- M1: clever if condition ------------------------- #
    # for k in range(m+n):
    #     # think: when should we put the elements of s1 to array?
    #     if (j>=n) or (i < m and s1[i] < s2[j]):
    #         s[k] = s1[i]
    #         i+=1
    #     else:
    #         s[k] = s2[j]
    #         j+=1
    # return s 
    
    # --------------------------- M2: standard solution -------------------------- #
    # while i < m and j < n:
    #     if s1[i] < s2[j]:
    #         s[i+j] = s1[i]
    #         i += 1
    #     else:
    #         s[i+j] = s2[j]
    #         j += 1
    # while i < m:
    #     s[i+j] = s1[i]
    #     i+=1
        
    # while j < n:
    #     s[i+j] = s2[j]
    #     j+=1
    # return s 
    
    # ------------------------------- M3: sentinel ------------------------------- #
    s1.append(float('inf'))
    s2.append(float('inf'))
    for k in range(m+n):
        if s1[i] < s2[j]:
            s[k] = s1[i]
            i+=1
        else:
            s[k] = s2[j]
            j+=1
    s1.pop()
    s2.pop()
    return s 

def merge_sort(a):
    # TODO
    # def helper(l, r):
    #     if l >= r: return 
    #     mid = (l+r)//2
    #     helper(l, mid)
    #     helper(mid+1, r)
    #     merge(a[l:mid+1], a[mid+1:r+1])

s1=[1,2,3]
s2=[-1,-2]
merge(s1,s2)


[-1, -2, 1, 2, 3]

## partition (in-place)

In [4]:
def partition(a, l, r, random_pivot=False):
    # l and r are inclusive
    i= l-1
    if random_pivot:
        rand_int = np.random.randint(l, r+1, None)
        a[rand_int], a[r] = a[r], a[rand_int]
        
    pivot = a[r]
    for j in range(l, r): # not including a[r], since it's pivot
        if a[j] <= pivot:
            i+=1
            a[i], a[j] = a[j], a[i]
    a[i+1], a[r] = a[r], a[i+1]
    return i+1

def quick_sort(a):
    def helper(l, r):
        if l >= r:
            return
        mid = partition(a, l, r)
        helper(l, mid-1)
        helper(mid+1, r)
    helper(0, len(a)-1)


def quick_select(a, k):
    # select the kth smallest
    k = k-1
    a = a.copy()
    def helper(l, r):
        mid = partition(a, l, r)
        if mid == k:
            return a[k]
        elif mid < k:
            return helper(mid+1, r)
        else:
            return helper(l, mid-1)
    return helper(0, len(a)-1)

            
# a=[2,1,3]
# a=[2, 8, 7, 1, 3, 5, 6, 4]
# a=[2, 2, 3, 3, 3, 3, 2, 1]
# partition(a, 0, len(a)-1)

# print(a)
# quick_sort(a)
# print(a)

# a = [1,2,3,4,5,6,7,8]
# a = [4,3,2,2,1]
a = np.random.permutation(5)
# a = [1,1,1,1,4,5,-1]
print(a)
quick_select(a, 1)
quick_select(a, 2)
# quick_select(a, 4)
# quick_select(a, 5)
# quick_select(a, 6)
# quick_select(a, 7)
# print(a)



[2 4 3 1 0]


0

1

# numerical algorithms

## newton's method for root finding

In [ ]:
# no need to supply
def newton(f, x0, epsilon=0.01, max_iter=100):
    xn = x0
    for _ in range(max_iter):
        f_xn = f(xn)
        if abs(f_xn) < epsilon:
            return xn
        # df_xn = df(xn)
        df_xn = sp.misc.derivative(f, xn)
        if df_xn == 0:
            print("derivative is zero... exiting")
            return None
        xn -= f_xn / df_xn

newton(lambda x: (x-3)*(x+1), x0=-2)
        
        
        
# need to supply derivative        
def newton(f, df, x0, epsilon=0.01, max_iter=100):
    xn = x0
    for _ in range(max_iter):
        f_xn = f(xn)
        if abs(f_xn) < epsilon:
            return xn
        df_xn = df(xn)
        if df_xn == 0:
            print("derivative is zero... exiting")
            return None
        xn -= f_xn / df_xn

newton(lambda x: x**2 - x - 1, lambda x: 2*x - 1, x0=10000)
        

/var/folders/c2/yj8ksjmn34v5f9csr1s82d8r0000gn/T/ipykernel_53516/529298773.py:9: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  df_xn = sp.misc.derivative(f, xn)


-1.0000102400262145

1.6195045111351039

In [ ]:
def newton(f,Df,x0,epsilon,max_iter):

    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> Df = lambda x: 2*x - 1
    >>> newton(f,Df,1,1e-8,10)
    Found solution after 5 iterations.
    1.618033988749989
    '''

    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None

        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

newton(lambda x: x**2 - x - 1, lambda x: 2*x - 1, x0=2, epsilon=0.1, max_iter=100)

Found solution after 2 iterations.


1.619047619047619

# statistics

## implement ma, expanding, ewm
- https://en.wikipedia.org/wiki/Exponential_smoothing
- in pandas, we use `alpha` instead of `beta` for smoothing factor
- the higher the smoothing factor, the less smooth
- remember: let `y_t` and `x_t` be the smoothed value and observed value at time t. Then,
- $y_t = \alpha x_{t} + (1-\alpha) y_{t-1} $
	- if `alpha = 1`, then `y_t` the same as `x_t` => the higher `\alpha`, the less smooth

In [33]:
def ewm(a, alpha=0.1):
    # assume data is a ndarray
    res = [0] * len(a)
    
    for i in range(len(a)):
        res[i] = a[0] if i == 0 else alpha * a[i] + (1-alpha) * res[i-1]
        
    return pd.DataFrame({
        'data': a,
        'ewm': res 
    })

def moving_average(a, window=3):
    # res[window-1] = average of first `window` entries 
    res = [np.nan] * len(a)
    # M1: naive
    # for i in range(window - 1, len(a)):
    #     res[i] = a[i-window+1:i+1].mean()
        
    # M2: fast
    cur_sum = a[:window].sum()
    res[window-1] = cur_sum/window
    for i in range(window, len(a)):
        cur_sum += - a[i-window] + a[i]
        res[i] = cur_sum/window
    
    return pd.DataFrame({
        'data': a,
        'ma': res 
    })

def expanding_average(a):
    cur_sum = 0
    res = a.copy()
    for i in range(len(a)):
        cur_sum += a[i]
        print(cur_sum)
        res[i] = cur_sum / (i+1)
        
    return pd.DataFrame({
        'data': a,
        'expanding_mean': res 
    })



# a = np.arange(100)
a = np.arange(3)
# ewm(data, alpha=0.1)

# moving_average(a)
expanding_average(a)

0
0
1
1
2
3


,data,expanding_mean
0,0,0
1,1,0
2,2,1


In [ ]:
# # old
# def ewm(data, beta=0.9, correct=False):
#     res = 0
#     for x in data:
#         # res *= beta
#         # res += (1-beta) * x
#         res = res*beta + (1-beta) * x
#     if correct:
#         res /= 1-(beta)**len(data)
#     return res
# np.random.seed(1)
# data = np.random.random(size=100)
# data.mean()
# ewm(data)
# ewm(data, correct=True)
    

0.48587792760014564

0.5234505224663181

0.5234644264137515

## moving average
- calculate
	- moving window average
	- expanding window average

In [ ]:
def exponential_moving_average(a, alpha=0.1, correct=False):
    res = [0] * len(a) 
    res[0] = a[0]
    for i in range(1, len(a)):
        res[i] = (1-alpha) * res[i-1] + alpha * a[i]
    return res 

def moving_average(a, s=3):
    # res[s-1] = average of first s entries 
    n = len(a)
    res = [np.nan] * n
    
    # ---------------------------------------------------------------------------- #
    cur_mean = sum(a[:s])/s
    res[s-1] = cur_mean
    # ---------------------------------------------------------------------------- #
    
    for i in range(s, n):
        cur_mean += (-a[i-s] + a[i])/s
        res[i] = cur_mean
    return res 

def weighted_moving_average(a, n=3):
    n = len(a)
    res = [np.nan] * n

def expanding_average(a):
    n = len(a)
    res = [0] * n
    cur_sum = 0
    for i in range(n):
        cur_sum += a[i]
        res[i] = cur_sum / (i+1)
    return res

a = [1,2,3,4,5,6,7,8]
# moving_average(a, s=3)
# expanding_average(a)
exponential_moving_average(a, alpha=0.5)

[1, 1.5, 2.25, 3.125, 4.0625, 5.03125, 6.015625, 7.0078125]

In [ ]:
def moving_average(a, s=3):
    n = len(a)
    # good. Avoid re-calculating the mean over and over
    res = [np.nan]*n
    
    # --------------------------------- base case -------------------------------- #
    cur_sum = sum(a[:s])
    res[s-1] = cur_sum / s
    # ---------------------------------------------------------------------------- #
    
    for i in range(s, n):
        cur_sum = cur_sum - a[i-s] + a[i]
        res[i] = cur_sum / s
    return res
    pass

def expanding_average(a):
    n = len(a)
    res = [0]*n
    cur_sum = 0
    for i in range(n):
        cur_sum += a[i]
        res[i] = cur_sum / (i+1)
    return res

def ema(a, alpha=0.8, correct=False):
    pass 

a = [1,2,3,4,5,6,7,8]
moving_average(a, s=6)
expanding_average(a)

[nan, nan, nan, nan, nan, 3.5, 4.5, 5.5]

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]

## find expected frequency of a table

In [ ]:
obs = np.array([[15, 45], [8, 32]])
margsums = stats.contingency.margins(obs)
margsums
(margsums[0] * margsums[1])/obs.sum() # broadcasting
# n = len(obs)

# [margsums[0][i] * margsums[1][j] for i in range(n) for j in range(n)]


# M2 
obs = np.array([[15, 45], [8, 32]])
expected = np.full_like(obs, 0)

row_sum = obs.sum(axis=0)
col_sum = obs.sum(axis=1)

n = len(obs)
for i in range(n):
    for j in range(n):
        expected[i,j] = row_sum[i] * col_sum[j]
        
expected/obs.sum()

[array([[60],
        [40]]),
 array([[23, 77]])]

array([[13.8, 46.2],
       [ 9.2, 30.8]])

## implement anova test for one-way classification

In [ ]:
class f_oneway():
    def __init__(self):
        pass
    def anova(self, *samples, alpha=0.05):
        self.samples = [np.array(sample) for sample in samples]
        self.n_gps = len(self.samples)
        self.n_samples, self.sums, self.sums_sqaured = np.zeros(self.n_gps), np.zeros(self.n_gps), np.zeros(self.n_gps)
        for i, sample in enumerate(self.samples):
            self.n_samples[i] =  len(sample)
            self.sums[i] =  sample.sum()
            self.sums_sqaured[i] =  (sample**2).sum()
        self.sst = sum(self.sums_sqaured) - sum(self.sums)**2 / sum(self.n_samples)
        self.sse = sum(self.sums_sqaured) - (self.sums**2 / self.n_samples).sum()
        self.ssa = self.sst-self.sse
        
        dfn, dfd = self.n_gps - 1, sum(self.n_samples) - self.n_gps
        f_dist = stats.f(dfn=dfn, dfd=dfd)
        self.msa = self.ssa/dfn
        self.mse = self.sse/dfd
        ts = self.msa/self.mse
        cv = f_dist.isf(alpha)
        p_value = f_dist.sf(ts)
        
        res = pd.DataFrame(
            {
            'SS': [self.ssa, self.sse],
            'df': [dfn, dfd],
            'mean_square': [self.msa, self.mse],
            'computed_F': [ts, None],
            'critical_value': [cv, None],
            'p_value': [p_value, None]
            },
            index=pd.Index(['Treatments', 'error'], name='variation')
        )
        
        res.loc['Total',['SS','df']]= res.sum(numeric_only=True, axis=0)
        return res
    def ci_means(self, alpha=0.02):
        pt_est = self.sums/self.n_samples
        se = (self.mse/self.n_samples)**0.5
        t = stats.t(df=self.n_samples.sum()-len(self.samples)).isf(alpha/2)
        me = t*se
        return pt_est - me, pt_est + me
    
        # print(f"{self.mse = }")
        # print(f"{self.n_samples = }")
        # print(f"{pt_est = }")
        # print(f"{se = }")
        # print(f"{t = }")
        # print(f"{self.n_samples.sum()-len(self.samples) = }")
        
        # print(self.sums)
        
        


# def f_oneway(*samples, alpha=0.05):
#     # this implementation uses numpy 
#     samples = [np.array(sample) for sample in samples]
#     n_gps = len(samples)
#     n_samples, sums, sums_sqaured = np.zeros(n_gps), np.zeros(n_gps), np.zeros(n_gps)
#     for i, sample in enumerate(samples):
#         n_samples[i] =  len(sample)
#         sums[i] =  sample.sum()
#         sums_sqaured[i] =  (sample**2).sum()
#     sst = sum(sums_sqaured) - sum(sums)**2 / sum(n_samples)
#     sse = sum(sums_sqaured) - (sums**2 / n_samples).sum()
#     ssa = sst-sse
    
#     dfn, dfd = n_gps - 1, sum(n_samples) - n_gps
#     f_dist = stats.f(dfn=dfn, dfd=dfd)
#     msa = ssa/(dfn)
#     mse = sse/(dfd)
#     ts = msa/mse
#     cv = f_dist.isf(alpha)
#     p_value = f_dist.sf(ts)
    
#     # res = {
#     #     'ts': ts,
#     #     'cv': cv,
#     #     'p_value': p_value
#     # }
    
    
#     res = pd.DataFrame(
#         {
#         'SS': [ssa, sse],
#         'df': [dfn, dfd],
#         'mean_square': [msa, mse],
#         'computed_F': [ts, None],
#         'critical_value': [cv, None],
#         'p_value': [p_value, None]
#         },
#         index=pd.Index(['Treatments', 'error'], name='variation')
#     )
    
#     res.loc['Total',['SS','df']]= res.sum(numeric_only=True, axis=0)
#     return res
    
# tillamook = [0.0571, 0.0813, 0.0831, 0.0976, 0.0817, 0.0859, 0.0735,
#              0.0659, 0.0923, 0.0836]
# newport = [0.0873, 0.0662, 0.0672, 0.0819, 0.0749, 0.0649, 0.0835,
#            0.0725]
# petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
# magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
#            0.0689]
# tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]
# f_oneway(tillamook, newport, petersburg, magadan, tvarminne) # F_onewayResult(statistic=7.121019471642447, pvalue=0.0002812242314534544)

A = [30, 26, 27, 29]
B = [28, 29, 28, 31]
C = [29, 32, 27, 31, 30]
D = [45, 35, 45, 37, 40]
# f_oneway(A,B,C,D, alpha=0.01)

test = f_oneway()
test.anova(A,B,C,D, alpha=0.01)
test.ci_means()





,SS,df,mean_square,computed_F,critical_value,p_value
variation,,,,,,
Treatments,476.5,3.0,158.833333,19.505848,5.563886,0.000029
error,114.0,14.0,8.142857,NaN,NaN,NaN
Total,590.5,17.0,NaN,NaN,NaN,NaN


(array([24.25541229, 25.25541229, 26.45073893, 37.05073893]),
 array([31.74458771, 32.74458771, 33.14926107, 43.74926107]))

## self-implementation of multiple linear regression
- find parameter estimates
- SE of estimates
- CI for parameters
- CI and PI for Y

In [ ]:
path = '/Users/whlin/Library/CloudStorage/OneDrive-HKUSTConnect/Documents/python/SEHH3164_Python/lecture notes/data/regression_1.csv'
df = pd.read_csv(path, index_col=0)
X, y = df.iloc[:, [0]], df.iloc[:, 1]
model = LinearRegression().fit(X,y)
model.intercept_
model.coef_


model = LinearRegression().fit(X*10,y)
model.intercept_
model.coef_

model = LinearRegression().fit(X,y*10)
model.intercept_
model.coef_


1.4464085156031823

array([0.01001084])

1.4464085156031823

array([0.00100108])

14.464085156031828

array([0.10010839])

In [ ]:
def prepare_data(data, add_const=True, y_name='y', to_numpy=False):
    # assume the given data (both X and y) is in dataframe
    # need to provide y_name
    if add_const:
        data.insert(0, 'const', 1)
    X, y = data.drop(y_name, axis=1), data[y_name]
    return (X.to_numpy(), y.to_numpy) if to_numpy else (X, y)
    
    # X = np.hstack([
    #     np.ones((data.shape[0],1)),
    #     data.drop('y', axis=1).to_numpy()
    #     ])
    # y = data[y_name].to_numpy()
    # # y = data.y.to_numpy()
    # return X, y

class Regression():
    def __init__(self):
        pass
    def find_coef(self):
        return np.linalg.inv(self.X.T@self.X)@self.X.T@y
    def find_error(self):
        # SST, SSR, SSE
        # MST, MSR, MSE
        res = {}
        n = self.n_samples
        res['sse'] = self.y.T@self.y - (self.X.T@self.y).T@self.coef
        # res['sst'] = self.y.T@(np.eye(n) - 1/n * np.ones((n,n)))@self.y
        res['sst'] = self.y.T@y - n*(self.y.mean()**2)
        res['ssr'] = res['sst'] - res['sse']
        
        res['mse'] = res['sse'] / self.df
        res['mst'] = res['sst'] / (n - 1)
        res['msr'] = res['ssr'] / (self.n_coef - 1)
        
        res['r_2'] = 1 - res['sse'] / res['sst']
        res['r_2_adjust'] = 1 - res['mse'] / res['mst']
        res['r'] = res['r_2']**0.5
        
        return res
        pass 
    
    def find_coef_se(self):
        # return pd.Series((self.mse * np.linalg.inv(self.X.T@self.X).diagonal())**0.5, index=self.feat_names)
        return (self.mse * np.linalg.inv(self.X.T@self.X).diagonal())**0.5
    
    def find_coef_ci(self, alpha=0.05, side='two', dp=None):
        t = stats.t(df=self.df).ppf(1-alpha/2)
        if dp:
            t = round(t, dp)
        me = t * self.coef_se
        return pd.DataFrame({
            'val': self.coef,
            'se': self.coef_se,
            'ci_l': self.coef - me,
            'ci_u': self.coef + me,
            't-statistic': (t_stats:=self.coef/self.coef_se),
            'p_value': (p_value:=2* stats.t(df=self.df).sf(np.abs(t_stats))),
            f'sig (at {alpha})': p_value <= alpha,
        }).set_axis(self.feat_names, axis=0)

    def predict(self, X):
        return X@self.coef

    def find_ci_pi_y(self, X_val, alpha=0.05, dp=None):
        pt_est = self.predict(X_val)
        X_inv = np.linalg.inv(self.X.T@self.X)
        t = stats.t(df=self.df).ppf(1-alpha/2)
        if dp:
            t = round(t, dp)
        var = self.mse * np.array([row.T @ X_inv @ row for row in X_val])
        se_ci, se_pi = var**0.5, (self.mse + var)**0.5
        me_ci, me_pi = t * se_ci, t * se_pi
        res = {
            'pred_E(Y)': pt_est,
            'se_ci': se_ci,
            'se_pi': se_pi,
            'ci_l': pt_est - me_ci,
            'ci_u': pt_est + me_ci,
            'pi_l': pt_est - me_pi,
            'pi_u': pt_est + me_pi,
        }
        return pd.DataFrame(res)
        
    def fit(self, X, y):
        # self.feat_names = X.columns
        if isinstance(X, pd.DataFrame):
            self.feat_names = X.columns
            self.X_df = X.copy()
            X = X.to_numpy()
        if isinstance(y, pd.Series):
            self.y_name = y.name
            y = y.to_numpy()
        
        self.X, self.y = X, y
        self.n_samples, self.n_coef = X.shape
        # print(self.n_samples, self.n_coef)
        self.df = self.n_samples - self.n_coef
        self.coef = self.find_coef()
        # self.sse = self.find_error()
        errors = self.find_error()
        for key, value in errors.items():
            # print(key, value)
            exec(f"self.{key} = {value}")
        # self.coef_se = (self.mse * np.linalg.inv(self.X.T@self.X).diagonal())**0.5
        self.coef_se = self.find_coef_se()
        self.coef_ci = self.find_coef_ci()
        return self
    
    def anova(self, alpha=0.05, dp=None):
        f_dist = stats.f(dfn=self.n_coef-1, dfd=self.df)
        ts = self.msr/self.mse
        cv = f_dist.isf(alpha)
        p_value = f_dist.sf(ts)
        # print(cv)
        res = pd.DataFrame(
            {
            # 'variation': ['regression', 'error'],
            'SS': [self.ssr, self.sse],
            'df': [self.n_coef-1, self.df],
            'mean_square': [self.msr, self.mse],
            'computed_F': [ts, None],
            'critical_value': [cv, None],
            'p_value': [p_value, None]
            },
            index=pd.Index(['regression', 'error'], name='variation')
        )
        
        res.loc['Total',['SS','df']]= res.sum(numeric_only=True, axis=0)
        # print(res)
        return res
    
    def show_stats_all_comb(self):
        res = {}
        for i in range(1, X.shape[1]):
            for comb in combinations(range(1, X.shape[1]), i):
                # print("hi")
                new_obj = Regression()
                # self.X_df.iloc[:, list(comb)].shape
                new_obj.fit(self.X_df.iloc[:, [0]+list(comb)],self.y)
                new_obj.find_error()
                res[comb] = new_obj.sse
        print(res)
    
    
    # def partial_anova(self, drop, alpha=0.05, ):
    #     partial = Regression().fit(X.drop())
        
    
path = '/Users/whlin/Library/CloudStorage/OneDrive-HKUSTConnect/Documents/python/SEHH3164_Python/lecture notes/data/regression_2.csv'
data = pd.read_csv(path).set_axis(['x_1','x_2','y'], axis=1)

X, y = prepare_data(data)
reg = Regression().fit(X, y)
reg.coef
reg.df
reg.sse

# x_val = np.array([[1,35,250],[2,35,250]])

# reg.find_coef_ci(alpha=0.01)
# reg.find_ci_pi_y(x_val)

# reg.anova()


reg.show_stats_all_comb()

# dir(reg)
# reg.y
# reg.coef_se
# reg.coef_ci




In [ ]:
np.random.seed(2)
n_samples = 2
n_coef = 4
X = pd.DataFrame(np.random.normal(size=(n_samples,n_coef)), columns=[f"x_{i+1}" for i in range(n_coef)])
y = X.sum(axis=1) + np.random.normal(loc=2, scale=4, size=n_samples)
X.insert(0, 'const', 1)

for i in range(1, X.shape[1]):
    for comb in combinations(range(1, X.shape[1]), i):
        X.iloc[:, [0] + list(comb)]

,const,x_1
0,1,-0.416758
1,1,-1.793436


,const,x_2
0,1,-0.056267
1,1,-0.841747


,const,x_3
0,1,-2.136196
1,1,0.502881


,const,x_4
0,1,1.640271
1,1,-1.245288


,const,x_1,x_2
0,1,-0.416758,-0.056267
1,1,-1.793436,-0.841747


...

,const,x_1,x_2,x_3
0,1,-0.416758,-0.056267,-2.136196
1,1,-1.793436,-0.841747,0.502881


,const,x_1,x_2,x_4
0,1,-0.416758,-0.056267,1.640271
1,1,-1.793436,-0.841747,-1.245288


,const,x_1,x_3,x_4
0,1,-0.416758,-2.136196,1.640271
1,1,-1.793436,0.502881,-1.245288


,const,x_2,x_3,x_4
0,1,-0.056267,-2.136196,1.640271
1,1,-0.841747,0.502881,-1.245288


,const,x_1,x_2,x_3,x_4
0,1,-0.416758,-0.056267,-2.136196,1.640271
1,1,-1.793436,-0.841747,0.502881,-1.245288


In [ ]:
def prepare_data(data, y_name='y'):
    # assume the given data (both X and y) is in dataframe
    # need to provide y_name
    X = np.hstack([
        np.ones((data.shape[0],1)),
        data.drop('y', axis=1).to_numpy()
        ])
    y = data[y_name].to_numpy()
    # y = data.y.to_numpy()
    return X, y

def find_estimates(X, y):
    return np.linalg.inv(X.T@X)@X.T@y

def find_SSE(X, y, beta, mse=False):
    sse = y.T@y - (X.T@y).T@beta
    return sse/(X.shape[0] - X.shape[1]) if mse else sse

def find_SST(y):
    n = len(y)
    return y.T@(np.eye(n) - 1/n * np.ones((n,n)))@y

def find_SSR(X, y, beta, mse=False):
    return find_SST(y) - find_SSE(X, y, beta, mse=False)

def ci_coeff(X, beta, mse, alpha=0.05):
    df = X.shape[0] - X.shape[1]
    t = stats.t(df=df).ppf(1-alpha/2)
    se = (mse * np.linalg.inv(X.T@X).diagonal())**0.5
    me = t*se
    
    return (
        pd.DataFrame([
            beta - me, 
            beta + me, 
            beta/se
            ])
        .T
        .set_axis(['ci_l', 'ci_u', 't-statistic'], axis=1)
        .set_axis([f'b_{i}' for i in range(X.shape[1])])
    )

def predict(X, beta):
    return X@beta

def ci_mean(x_val, X, beta, mse, alpha=0.05):
    # TODO: generalize x_val to matrix
    df = X.shape[0] - X.shape[1]
    pt_est = predict(x_val,beta)
    print(pt_est)
    se_ci = (mse * x_val.T@np.linalg.inv(X.T@X)@x_val)**0.5
    se_pi = (mse * (1+x_val.T@np.linalg.inv(X.T@X)@x_val))**0.5
    t = stats.t(df=df).ppf(1-alpha/2)
    me_ci = t*se_ci
    me_pi = t*se_pi
    return pd.DataFrame(
        [[pt_est - me_ci, pt_est + me_ci, pt_est - me_pi, pt_est + me_pi]], 
        columns=['ci_l','ci_u','pi_l','pi_u']
        )
    

# path = '/Users/whlin/Library/CloudStorage/OneDrive-HKUSTConnect/Documents/python/SEHH3164_Python/lecture notes/data/regression_1.csv'
# data = pd.read_csv(path).drop('Store',axis=1).set_axis(['x_1','y'], axis=1)
# data

path = '/Users/whlin/Library/CloudStorage/OneDrive-HKUSTConnect/Documents/python/SEHH3164_Python/lecture notes/data/regression_2.csv'
data = pd.read_csv(path).set_axis(['x_1','x_2','y'], axis=1)

X, y = prepare_data(data)

beta = find_estimates(X,y)
find_SSE(X, y, beta, mse=False)
find_SST(y)
find_SSR(X, y, beta, mse=False)

# beta
# print(X.shape[0] - X.shape[1])
# mse = find_SSE(X,y,beta, mse=True)
# y@y
# y.T@y
# ci_coeff(X, beta, mse, alpha=0.1)

# predict(X, beta)

# x_val = np.array([1,35,250])
# ci_mean(x_val, X, beta, mse, alpha=0.02)
# ci_mean(X, X, beta, mse)

256.2843019978609

2020.499999999997

1764.2156980021362

# machine learning

## leave-one-out method to do prediction (intact interview)
- Given a list L of values generated independently by some unknown process, we will use the mean of L to predict unseen values generated by the same process.
- Use leave-one-out cross-validation to estimate the mean absolute error of this process.
- Input: An array of floats arr
- Output: A float score
Example:
- arr = [1 2 3]
- score = 1.0
- mean = 3
- for 1:
	- |2.5 - 1| = 1.5
- for 2:
	- |2 - 2| = 0
- for 3:
	- |1.5 - 3| = 1.5
- so, score = (1.5+0+1.5)/3 = 1

In [ ]:
def k_fold(a, metric='mae'):
    n = len(a)
    tot = sum(a)
    
    # M1
    # error = 0 
    # for x in a:
    #     error += abs((tot-x)/(n-1) - x)
    # return error/n
    
    # M2:
    errors = [abs((tot-x)/(n-1) - x) for x in a]
    return sum(errors)/len(errors)

    # sum_ = sum(a)
    # n = len(a)
    
    # pred = np.array([(sum_ - x) / (n-1) for x in a])
    
    # # pred = [0]*n
    # # for i, x in enumerate(a):
    # #     pred[i] = (sum_ - x) / (n-1)
    
    # return np.abs(np.array(pred) - np.array(a)).mean()

a = [1,2,3]
k_fold(a)

1.0

## create bag-of-word matrix
Write a function to obtain bag-of-word feature matrix to train a text classification model. The input is a list of strings texts, where each string represents one training item (one text). The function should return a matrix where each row represents one training item, and each column represents one word. Words are ordered alphabetically, i.e., the first column corresponds to the first word in alphabetic order.

Specifications:
- Tokenize on whitespace (assume that only alphabetical characters are included)
- Convert all tokens to lowercase
- Use simple word counts
Example:
- Input: texts = ["This is an example", "that", "an example", "This is that that"]
- Output: bow_matrix =[[1, 1, 1,0, 1],[0,0, 0, 1,0], [1, 1,0,0,0],[0,0, 1, 2, 1 ]]

- idea: first find all the unique words first
- then, sort the list, and then create a dict whose item is (word, index)

In [ ]:
def bow(texts):
    # M1:
    # words = {word for text in texts for word in text.lower().split(' ')}
    # words_to_order = dict(zip(sorted(words), range(len(words))))
    # res = [[0]*len(words)for i in range(len(texts))]
    
    # for i, text in enumerate(texts):
    #     for word in text.lower().split(' '):
    #         res[i][words_to_order[word]] += 1
    # return res 

    # M2:
    texts_formatted = [text.lower().split(' ') for text in texts]
    words = sorted({s for text in texts_formatted for s in text})
    n_words = len(words)
    words = dict(zip(words, range(n_words)))
    
    # ---------------------- create the required array first --------------------- #
    # res = [ [0]*n_words for i in range(len(texts)) ] 
    # for i, text in enumerate(texts_formatted):
    #     for word in text:
    #         res[i][words[word]] += 1
    # ---------------------------------------------------------------------------- #
    
    # ----------------------- create each row sequentially ----------------------- #
    res = []
    for text in texts_formatted:
        arr = [0] * n_words
        for word in text:
            arr[words[word]] += 1
        res.append(arr)
    # ---------------------------------------------------------------------------- #
    
    return res
texts = ["This is an Example", "that", "an example", "This is that that"]
bow(texts)

[[1, 1, 1, 0, 1], [0, 0, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 1, 2, 1]]

## svd (singular value decomposition)

In [ ]:
# X = np.array([[1,1,1],[2,2,2],[3,3,3]])
# X = np.array([[1,0],[1,1]])
X = np.array([[1,1],[1,1],[0,0]])
U, sigma, V = np.linalg.svd(X)
U, sigma, V

sig = np.zeros_like(X)
sig[:X.shape[1], :X.shape[1]] = np.diag(sigma)

U.shape
sig.shape
V.shape


U @ sig @ V


# U, sigma, V
# U@sigma

(array([[-0.70710678, -0.70710678,  0.        ],
        [-0.70710678,  0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 array([2.00000000e+00, 3.35470445e-17]),
 array([[-0.70710678, -0.70710678],
        [ 0.70710678, -0.70710678]]))

(3, 3)

(3, 2)

(2, 2)

array([[1., 1.],
       [1., 1.],
       [0., 0.]])

## distance matrix

In [188]:
np.random.seed(1)
def distance_matrix(X):
    m,n=X.shape
    dist = np.zeros((m,m))
    for i in range(m):
        for j in range(i, m):
        # for j in range(m):
            dist[i,j] = np.linalg.norm(X[i] - X[j])
    print(dist)


m,n = 5,2
X = np.arange(5*2).reshape(5, 2)
print(X)
# [[0 1]
#  [2 3]
#  [4 5]
#  [6 7]
#  [8 9]]
print(distance_matrix(X))
# [[ 0.          2.82842712  5.65685425  8.48528137 11.3137085 ]
#  [ 2.82842712  0.          2.82842712  5.65685425  8.48528137]
#  [ 5.65685425  2.82842712  0.          2.82842712  5.65685425]
#  [ 8.48528137  5.65685425  2.82842712  0.          2.82842712]
#  [11.3137085   8.48528137  5.65685425  2.82842712  0.        ]]

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[ 0.          2.82842712  5.65685425  8.48528137 11.3137085 ]
 [ 0.          0.          2.82842712  5.65685425  8.48528137]
 [ 0.          0.          0.          2.82842712  5.65685425]
 [ 0.          0.          0.          0.          2.82842712]
 [ 0.          0.          0.          0.          0.        ]]
None


## K-means

In [97]:
np.random.seed(1)
def k_means(data, n_clusters=8, max_iter=100, tol=0.0001):
    m,n = data.shape
    rand_nums = np.random.randint(0, m, n_clusters)
    # print(rand_nums)
    centroids = data[rand_nums]
    # print(centroids)
    assignments = np.full(m, -1)
    dist = np.zeros((m,n_clusters))
    
    # print(data)
    
    # print((202-61)**2 + (300+765)**2 + (278-932)**2)
    # dist[:, 0] = ((data-centroids[0])**2).sum(axis=1)
    
    # cluster assignment step 
    for j in range(max_iter):
        for i in range(n_clusters):
            dist[:, i] = ((data - centroids[i])**2).sum(axis=1)
        new_assignments = dist.argmin(axis=1)
        # print(f"{new_assignments = }")
        # print(f"{assignments = }")
        if (assignments == new_assignments).all():
            print(f"converged at iteration {j}")
            # print(assignments)
            break
        assignments = new_assignments
        
        # print(f"{centroids = }")
        # cluster recalculation step 
        for i in range(n_clusters):
            centroids[i] = data[assignments==i].mean(axis=0)
    
        # print(f"{centroids = }")
    
    
    # print(dist)
    # print(dist.shape)
    
    pass

m = 10000
n = 30
n_clusters=2
data=np.random.randint(-1000,1000, size=(m,n))
# data
k_means(data, n_clusters=n_clusters)

# ptr = np.array([1,2,3,4])

# ((data-ptr)**2).sum(axis=1)

# data[np.random.randint(0, n, n_clusters)]

converged at iteration 69


In [87]:
m = 20
n = 3
data=np.random.randint(-1000,1000, size=(m,n))
data.mean(axis=0)
# data.mean(axis=0)

array([-54.4 , 136.9 ,  87.95])

## PCA

In [ ]:
def pca(X, n_components=1, rowvar=True):
    # - find covariance matrix
    # - find the eignvalues and eigenvectors of the covariance matrix 
    # - do X @ eig_vec to find the scalar projection of each vector onto the space spanned by the eigenvectors
    # - if want to decide the dimension of the reduced vector by the "variance explained way", we need to find the eigenvalaues too 

    cov = np.cov(X, rowvar=False, ddof=0) # this will do the mean centering 
    eig_val, eig_vec = np.linalg.eig(cov) 

    # ------------------------- covariance matrix by hand ------------------------ #
    # Y = X - X.mean(axis=0, keepdims=1)
    # cov = Y.T@Y/4
    # ---------------------------------------------------------------------------- #
    
    eig_vec = eig_vec[:, :n_components]
    
    X_reduce = X @ eig_vec
    X_approx = X_reduce @ eig_vec.T
    
    # print(X_reduce)
    # print(X_approx)
    
    # print(np.trace(eig_val))
    # print(np.sum(eig_val))
    # print(np.trace(cov))
    
    
    # print(eig_val)
    # eig_vec @ X_reduce.T
    # X_reduce @ eig_vec.T
    # X_demean = X - np.mean(axis=0)
    # print(pd.Series(eig_val).expanding().mean())
    
    # print(pd.Series(eig_val).expanding().sum())
    
    # pd.Series(eig_val)
    # print(pd.Series((eig_val/eig_val.sum()).cumsum()), axis=)
    
    return X_reduce, X_approx

X = np.array([[3,3],[0,2], [-1,-1], [2,0]])
X
pca(X, n_components=2)

array([[ 3,  3],
       [ 0,  2],
       [-1, -1],
       [ 2,  0]])

(array([[ 4.24264069, -0.        ],
        [ 1.41421356,  1.41421356],
        [-1.41421356,  0.        ],
        [ 1.41421356, -1.41421356]]),
 array([[ 3.,  3.],
        [ 0.,  2.],
        [-1., -1.],
        [ 2.,  0.]]))

## covariance matrix
- case 1: given a matrix, want to find the covariance of each columns
- NOTE: `cor[i,j] = cov[i,j]/( cov[i,i] * cov[j,j] )`

In [ ]:
def cov(x, y=None, corr=False, row_wise=False):
    if row_wise:
        return cov(x.T, y=y, corr=corr, row_wise=False)
    
    if y is not None:
        return cov(np.column_stack((x,y)), corr=corr)
    
    n_cols = x.shape[1]
    x_demean = x - x.mean(axis=0)
    # covar = (x_demean.T @ x_demean) / len(x_demean)
    covar = (x_demean.T @ x_demean) / x_demean.shape[0]
    
    if not corr:
        return covar
    
    corr = np.array(
        [
            [covar[i,j]/np.sqrt(covar[i,i] * covar[j,j] ) for j in range(n_cols)] 
            for i in range(n_cols)
        ]
        )
    
    return corr

np.random.seed(1)
n_rows = 10
n_cols = 4

data = np.random.random((n_rows, n_cols))
# data
# x = np.linspace(-10, 10, 10)
# y = x**2
# y = -10*x + 100
# cov(x,y, corr=True)

np.corrcoef(data, rowvar=False)

# cov(data)
# np.cov(data, rowvar=False, ddof=0)
# np.corrcoef(x,y)


# data = np.arange(10*4).reshape(10,4)
# cov(data, corr=True)
# cov([1,2,3],[4,5,6])


# # data = np.random.random((n_rows, n_cols))
# data_demean = data-data.mean(axis=0)

# cov = (data_demean.T @ data_demean) / len(data_demean)
# print(cov)
# cor = np.full((n_cols, n_cols), 0.0)
# for i in range(n_cols):
#     for j in range(n_cols):
#         cor[i,j] = cov[i,j]/np.sqrt(cov[i,i] * cov[j,j] )
# cor

# base = np.random.random(n_rows)
# data = np.array([base + stats.norm(scale=0.01).rvs(n_rows) for _ in range(n_cols)]).T
# data_demean = data - data.mean(axis=0)

# data_demean.T @ data_demean / data_demean.shape[0]

# np.cov(data.T, ddof=0)

# a1 = np.array([0, 1, 1])
# a2 = np.array([2, 2, 1])

# np.cov(a1, a2, ddof=0)

# data = np.array([a1,a2]).T
# data_demean = data-data.mean(axis=0)
# data_demean.T @ data_demean / 3


array([[ 1.        ,  0.32321673,  0.45383935, -0.14576703],
       [ 0.32321673,  1.        , -0.32483296,  0.23901403],
       [ 0.45383935, -0.32483296,  1.        ,  0.03381041],
       [-0.14576703,  0.23901403,  0.03381041,  1.        ]])

## standardization/normalization etc
- StandardScaler, Normalizer (i,e, the norm of each feature is 1), MinMaxScaler, proportion scaling
- in general, normalization by column is much easier...

### summary of all scaling methods

In [ ]:
# df = tips.select_dtypes('number')
a,b = 10, 15
df = pd.DataFrame(
     [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
)

mean_ = df.mean()
min_ = df.min()
max_ = df.max()
std_ = df.std(ddof=0)
sum_ = df.sum()
median_ = df.median()
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)

# standard scaler
df.subtract(mean_).div(std_).head(2)

# minmax scaler
df.subtract(min_).div(max_ - min_) .head(2)

# maxabs scaler
df.div(max_.abs()) 

# length normalizer, so that the norm of each column is one unit 
# df.div(df.pow(2).sum().pow(0.5))
df.div(np.linalg.norm(df, axis=0))

# proportion scaler
df.div(sum_)

# AB scaler (i.e., all entries are from a to b)
a + df.subtract(min_).div(max_ - min_) * (b - a)

# robust scaler: remove median and then scale the result by IQR
df.subtract(median_).div(q3-q1)

### normalize each row of a df

In [ ]:
# idea: first find the sum of each row, then divide each entry by this row sum
df = pd.DataFrame(np.arange(3*4).reshape(3,4))
row_sums = df.sum(axis=1)
df
row_sums

# M1: use div
df.div(row_sums, axis=0).sum(axis=1)

# M2: use apply
df.apply(lambda s: s/row_sums).sum(axis=1)

# M3: bad...
temp = pd.concat([row_sums.to_frame()]*4, axis=1)
temp.columns = [0,1,2,3]
temp
(df/temp).sum(axis=1)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


0     6
1    22
2    38
dtype: int64

0    1.0
1    1.0
2    1.0
dtype: float64

0    1.0
1    1.0
2    1.0
dtype: float64

,0,1,2,3
0,6,6,6,6
1,22,22,22,22
2,38,38,38,38


0    1.0
1    1.0
2    1.0
dtype: float64

# evaluation metrics

## entropy, cross entropy, KL divergence

- $\mathrm{H}(X):=-\sum_{x \in \mathcal{X}} p(x) \log p(x)=\mathbb{E}[-\log p(X)]$
	- entropy of a signle distribution
- $D(P || Q) = \sum_i p_i \log\frac{p_i}{q_i}$
	- relative entropy / KL divergence
	- remember, relative entropy the same as KL divergence!!!
- $CE(P, Q) = -\sum_i p_i \log q_i$ 
	- cross entropy



In [7]:
# M1: self-defined function
def entropy(x, y=None, type='relative', base=2):
    if y is None: # do entropy on one single distrbution only
        return -np.sum(x*np.log2(x))
    else:
        if type == 'relative':
            return np.sum(x * np.log2(x/y))
        elif type == 'cross':
            return -np.sum(x * np.log2(y))

x = np.array([0.1,0.3,0.3,0.1,0.2])
y = np.array([0.1,0.1,0.5,0.1,0.2])

stats.entropy(x, base=2)
entropy(x,y, base=2) # same as stats.entropy(x,y, base=2)
entropy(x,y, base=2, type='cross')


# M2: use stats.entropy function
stats.entropy(x, base=2) # entropy of single distribution
stats.entropy(x,y, base=2) # relative entropy
entropy(x, base=2) + entropy(x,y, base=2) # cross entropy

2.1709505944546685

0.2543990719664849

2.4253496664211536

2.1709505944546685

0.2543990719664849

2.425349666421154

In [8]:
# ------------------------------ further example ----------------------------- #
n = 10
rv = stats.binom(p=0.3, n =10) 
stats.entropy(rv.pmf(range(n+1)), base=np.exp(1))
rv.entropy()

rv2 = stats.binom(p=0.2, n =10) 
stats.entropy(rv.pmf(range(n+1)), rv2.pmf(range(n+1)), base=np.exp(1)) # D(P || Q)
stats.entropy(rv2.pmf(range(n+1)), rv.pmf(range(n+1)), base=np.exp(1)) # D(Q || P)

1.7790787840900633

1.779078784090063

0.2816755759528351

0.257320924779852

## similarity measure
- cosine
- covariance
- correlation

In [ ]:
def cosine_similarity(x,y):
    return x@y/(np.linalg.norm(x) * np.linalg.norm(y))

def covariance(x,y):
    return (x-x.mean())@(y-y.mean()) / len(x)

def correlation(x,y):
    # return covariance(x,y)/(x.std(ddof=0) * y.std(ddof=0))
    
    # M2
    x_demean = x-x.mean()
    y_demean = y-y.mean()
    return (x_demean @ y_demean) / np.sqrt(np.power(x_demean, 2).sum() * np.power(y_demean, 2).sum())

x = np.array([1,2,3])
y = np.array([4,5,6])
cosine_similarity(x,y)
covariance(x,y)
correlation(x,y)

0.9746318461970762

0.6666666666666666

1.0

## metrics for regression

In [111]:
# regression
def mean_squared_error(y_true, y_pred):
    # related to L2 norm
    # return ((y_true-y_pred)**2).mean()
    return np.power(y_true-y_pred, 2).mean()
    # return np.linalg.norm(y_true-y_pred)**2/len(y_true)
    pass
def mean_absolute_error(y_true, y_pred):
    # related to L1 norm
    return np.abs(y_true-y_pred).mean()
    # return np.linalg.norm(y_true-y_pred, ord=1)/len(y_true)
    pass
def max_error(y_true, y_pred):
    # same as L_inf norm
    return np.abs(y_true-y_pred).max()
    # return np.linalg.norm(y_true-y_pred, ord=np.inf)
def median_absolute_error(y_true, y_pred):
    return np.median(np.abs(y_true-y_pred))
    pass
def r2_score(y_true, y_pred):
    # remember: SST = SSE + SSR
    SSE = np.power(y_true - y_pred, 2).sum()
    SST = np.power(y_true - y_true.mean(), 2).sum()
    return 1 - SSE/SST
    pass
def mean_absolute_percentage_error(y_true, y_pred):
    return (np.abs(y_true-y_pred) / np.abs(y_true)).mean()
    pass
def mean_squared_log_error(y_true, y_pred):
    pass


y_true = np.array([1,2,3])
# y_pred = np.array([1,2,100])
y_pred = np.array([2,2,2])

mean_squared_error(y_true, y_pred)
mean_absolute_error(y_true, y_pred)
median_absolute_error(y_true, y_pred)
mean_absolute_percentage_error(y_true, y_pred)
max_error(y_true, y_pred)
r2_score(y_true, y_pred)
# classification

0.6666666666666666

0.6666666666666666

1.0

0.4444444444444444

1.0

0.0

## metrics for classification 

In [42]:
def accuracy(cm):
    (tn, fp), (fn, tp) = cm
    return (tn + tp) / cm.sum()
    # return (cm[0,0] + cm[1,1]) / cm.sum()
def error_rate(cm):
    return 1 - accuracy(cm)
def precision(cm):
    (tn, fp), (fn, tp) = cm
    return tp / (tp + fp)
    # return cm[1,1] / cm[:, 1].sum()
def recall(cm):
    # same as tpr 
    (tn, fp), (fn, tp) = cm
    return tp / (tp + fn)
    # return cm[1,1] / (cm[1,1] + cm[1,0])
def f_score(cm):
    return 2 / (1/recall() + 1/precision())
def tnr(cm):
    # same as specificity
    (tn, fp), (fn, tp) = cm
    return tn / (tn + fp)
    # return cm[0,0] / (cm[0,0] + cm[0,1])
def tpr(cm):
    # same as recall
    (tn, fp), (fn, tp) = cm
    return tp / (tp + fn)
    # return cm[1,1] / (cm[1,1] + cm[1,0])
def fnr(cm):
    (tn, fp), (fn, tp) = cm
    return fn / (fn + tp)
    # return cm[1,0] / (cm[1,0] + cm[1,1])
def fpr(cm):
    (tn, fp), (fn, tp) = cm
    return fp / (fp + tn)
    # return cm[0,1] / (cm[0,1] + cm[1,0])

# def confusion_matrix_word(y_true, y_pred):
#     cm = confusion_matrix(y_true, y_pred)
#     return {
#     'tn': cm[0,0],
#     'fp': cm[0,1],
#     'fn': cm[1,0],
#     'tp': cm[1,1],
#     }

def confusion_matrix(y_true, y_pred):
    cm = np.full((2,2),0)
    for true, pred in zip(y_true, y_pred):
        cm[true, pred] += 1
    return cm

y_true = [1]*4 + [0]*8
y_pred = [0,1,1,0,1,0,0,1,1,0,0,1]
cm = confusion_matrix(y_true, y_pred)
# print(cm)
# accuracy(cm)
# error_rate(cm)
# precision(cm)
tnr(cm)

# cm
# for x, y in cm:
#     print(x, y)

0.5

In [43]:
# # y_true = [0, 0, 1, 0, 1, 1]
# # y_pred = [0, 1, 1, 0, 0, 0]
# y_true = [1]*4 + [0]*8
# y_pred = [0,1,1,0,1,0,0,1,1,0,0,1]
# cm = np.full((2,2), 0)
# for true, pred in zip(y_true, y_pred):
#     # if true == pred == 0:
#     cm[true, pred] += 1

# p = cm[1,1] / cm[:,1].sum()
# r = cm[1,1] / cm[1,:].sum()
# # tpr = 

# cm
# cm_rate = cm/cm.sum(axis=1, keepdims=True)
# cm_rate

# tnr = specificity = cm_rate[0,0]
# fpr = cm_rate[0,1]
# fnr = cm_rate[1,0]
# tpr = recall = cm_rate[1,1]
# accuracy = cm.trace() / cm.sum()
# error_rate = 1 - accuracy
# precision = cm[1,1] / cm[:,1].sum()

## balanced_accuracy_score
In the binary case, balanced accuracy is equal to the arithmetic mean of sensitivity (true positive rate) and specificity (true negative rate), or the area under the ROC curve with binary predictions rather than scores:
- can be used for both multiclass and binary class

In [169]:
from sklearn.metrics import balanced_accuracy_score
y_true = [0, 1, 0, 0, 1, 0]
y_pred = [0, 1, 0, 0, 0, 1]

y_true = np.random.randint(2, size=100)
y_pred = np.random.randint(2, size=100)

# y_true
# y_pred

balanced_accuracy_score(y_true, y_pred)
roc_auc_score(y_true, y_pred)

# (recall_score(y_true, y_pred) + precision_score(y_true, y_pred))/2
(recall_score(y_true, y_pred) + recall_score(y_true, y_pred, pos_label=0))/2

0.4623397435897436

0.4623397435897436

0.4623397435897436

## top-k metrics

In [139]:
import numpy as np
from sklearn.metrics import top_k_accuracy_score
y_true = np.array([0, 1, 2, 2])
y_score = np.array([[0.5, 0.2, 0.2],  # 0 is in top 2
                    [0.3, 0.4, 0.2],  # 1 is in top 2
                    [0.2, 0.4, 0.3],  # 2 is in top 2
                    [0.7, 0.2, 0.1]]) # 2 isn't in top 2
top_k_accuracy_score(y_true, y_score, k=2)

# Not normalizing gives the number of "correctly" classified samples
top_k_accuracy_score(y_true, y_score, k=2, normalize=False)

0.75

3

# array problem

## compression of vector (intact interview)
You want to compress an array that often contains consecutive appearances of the same integer.
The task is to reduce the array into a compressed array that contains, one after the other, the integer seen and the number of consecutive repetitions of that integer.
Example:
Input: arr = [1, 1, 1, 5, 3, 3, 9, 3, 3, 3, 3, 3]
Output: [1, 3, 5, 1, 3, 2, 9, 1, 3, 5]

- Input: `[1,1,1,5,3,3,9,3,3,3,3,3]`
- Output: `[1,3,5,1,3,2,9,1,3,5]`
- explaination: in `output`: 
	- 1 appears 3 times in a row
	- 5 appears 1 time in a row
	- 3 appears 2 times in a row
	- 9 appears 1 time in a row
	- 3 appears 5 times in a row
- should be solved with 2 pointers




In [ ]:
def compress(a):
    # bad method 
    # res, n, i = [], len(a), 0
    # while i < n:
    #     val = a[i]
    #     cnt = 1
    #     j = i+1
    #     while j< n:
    #     # for j in range(i+1, n):
    #         if a[j] == val:
    #             cnt+=1
    #             j+=1
    #         else:
    #             break 
    #     i=j
    #     res.append(val)
    #     res.append(cnt)
    # return res 
    
    try:
        cur = a[0]
    except IndexError: # the array may be empty
        return []
    count, res = 1, []
    
    for x in a[1:]:
        if cur == x: # the next number is the same, so move forward
            count+=1 
        else:
            res.extend([cur, count])
            count = 1
            cur = x
    res.extend([cur, count])
    return res

def de_compress(a):
    res = []
    for i in range(0, len(a), 2): # good way to loop two elements in each step
        res.extend([a[i]]*a[i+1])
    return res
    
    # M2: doesn't work, since the result will be a list of lists
    # res = [[a[i]]*a[i+1] for i in range(0, len(a), 2)]
    # return res


# from gpt
def compress_array(arr):

    if not arr:
        return []

    result = [arr[0], 1]

    current = arr[0]

    count = 1

    for i in range(1, len(arr)):

        if arr[i] == current:

            count += 1

        else:

            result.append(count)

            current = arr[i]

            result.append(current)

            count = 1

    result.append(count)

    return result

# a = [1,1,1,5,3,3,9,3,3,3,3,3]
# a = [1,1,1,5,3,3,9,3,3,3,3,3,4]
# a = []
# [1,3,5,1,3,2,9,1,3,5]
# compress(a)
# a = [1,3,5,1,3,2,9,1,3,5]
# de_compress([1,3,5,1,3,2,9,1,3,5])
compress(a)
# a = [1,10]
# de_compress(a)

# compress_array(a)


[1, 3, 5, 1, 3, 2, 9, 1, 3, 5, 4, 1]

## compression of vector 2: `a[i] = number of times value i appears in total`
- a = [1,1,1,2,3,1,2,4,1]
- output: [0, 5, 2, 1, 1], since 0 appears 0 times, 1 appears 5 times, and so on.

In [ ]:
def compress_by_max(a):
    # M1: need to find max(a) first. Slow
    res = [0]* (max(a)+1)
    for x in a:
        res[x] += 1
    return res

    # M2: doesn't work. only return a dict
    # res = {}
    # for x in a:
    #     res[x] = res.get(x,0) + 1
    # return res
    
def decompress_by_max(a):
    # M1:
    res = []
    for i, x in enumerate(a):
        res.extend([i]*x)
    return res

    # M2:
    # cur = 0 
    # res = [0]*sum(a)
    # for i in range(len(a)):
    #     res[cur:cur+a[i]] = [i] * a[i]
    #     cur += a[i]
    # return res


# a = [1,1,1,2,3,1,2,4,1]
# compress_by_max(a)
decompress_by_max([5,1,2,1])

[0, 0, 0, 0, 0, 1, 2, 2, 3]

## implement combination(a,k)

## 392. Is Subsequence
Given two strings s and t, return true if s is a subsequence of t, or false otherwise.

A subsequence of a string is a new string that is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters. (i.e., "ace" is a subsequence of "abcde" while "aec" is not).

idea: let a pointer point to the current character of s, then iterate t.
- if the current of t == current of s, move the pointer of s
- if the pointer of s move to len(s), then True

In [6]:
def is_valid_subsequence(s, t):
    # check if s is a subsequence of t
    i = 0 
    for x in t:
        if s[i] == x:
            i+=1
            if i == len(s):
                return True
    return False 

    # M2
    # arr_idx = seq_idx = 0
    # while arr_idx < len(array) and seq_idx < len(sequence):
    #     if array[arr_idx] == sequence[seq_idx]:
    #         seq_idx+=1
    #     arr_idx+=1
    # return seq_idx == len(sequence)
    
s, t = "abc", "ahbgdc"
s, t = "axc", "ahbgdc"
is_valid_subsequence(s,t)

False

## 5. Longest Palindromic Substring


In [383]:
def longest_palindrome(s) -> str:
    n = len(s)
    n_res = 0
    res = []
    for i in range(n):
        for j in range(i, n):
            candidate = s[i:j+1]
            if all(x==y for x,y in zip(candidate, reversed(candidate))) and j-i+1 > n_res:
                n_res = j-i+1
                res = candidate
    return res, n_res

s = "abc"
longest_palindrome(s)

('a', 1)

## array rotation
- notes:
	- should do inplace
	- if d > len(a), then we don't need to repeat again, it's like 370 degree => 10 degree
	- so, do d % len(a)
	- for left rotate, the final position of a[i] = a[(i+d)%n]
	- smart way: divide the array into two halves, then use `+`

In [229]:
def rotate_left(a, d):
    # naive
    # n = len(a)
    # for i in range(d%n):
    #     temp = a[0]
    #     for j in range(n-1):
    #         a[j] = a[j+1]
    #     a[n-1] = temp 
    # return a

    # M2
    # n = len(a)
    # temp = a.copy()
    # for i in range(n):
    #     a[i] = temp[(i + d)%n]
    # return a

    # M3: smartest way
    n = len(a)
    d = d%n
    return a[d:] + a[:d]

def rotate_right(a, d):
    n = len(a)
    d = d%n
    return a[-d:] + a[:-d]
    pass

a = list(range(1, 11))
rotate_right(a, 3)

# a = [1,2,3,4,5]
rotate_left(a, 1001)

[8, 9, 10, 1, 2, 3, 4, 5, 6, 7]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 1]

## hackerrank: table of content

In [233]:
def table_of_contents(text):
    # Write your code here
    cur_chapter = 0
    cur_section = 0
    res = []
    for s in text:
        if s.startswith('#'):
            sec, content = s.split(' ',1)
            if len(sec) == 1: # new chapter
                cur_chapter += 1
                cur_section = 0 
                res.append(f"{cur_chapter}. {content}")
            elif len(sec) == 2: # new section
                cur_section+=1
                res.append(f"{cur_chapter}.{cur_section}. {content}")
    return res 

text = ['# Algorithms',
'This chapter covers the most basic algorithms.',
'## Sorting',
'Quicksort is fast and widely used in practice',
'Merge sort is a deterministic algorithm',
'## Searching',
'DFS and BFS are widely used graph searching algorithms',
'Some variants of DFS are also used in game theory applications',
'# Data Structures',
'This chapter is all about data structures',
"It's a draft for now and will contain more sections in the future",
'# Binary Search Trees',]

table_of_contents(text)

['1. Algorithms',
 '1.1. Sorting',
 '1.2. Searching',
 '2. Data Structures',
 '3. Binary Search Trees']

## hackerrank: find smallest divisor
Given two strings, s and t, create a function that operates per the following rules:

Find whether string s is divisible by string t.  String s divisible by string t if string t can be concatenated some number of times to obtain the string s.
If s is divisible, find the smallest string, u, such that it can be concatenated some number of times to obtain both s and t.
If it is not divisible, set the return value to -1.
Return the length of the string u or -1.

Example 1

s = 'bcdbcdbcdbcd'

t = 'bcdbcd'


If string t is concatenated twice, the result is 'bcdbcdbcdbcd' which is equal to the string s.  The string s is divisible by string t. 

Since it passes the first test, look for the smallest string, u, that can be concatenated to create both strings s and t.

The string 'bcd' is the smallest string that can be concatenated to create both strings s and t. 

The length of the string u is 3, which is the integer value to return.


Example 2

s = "bcdbcdbcd"

t = "bcdbcd"


If string t is concatenated twice, the result is "bcdbcdbcdbcd" which is greater than string s.  There is an extra "bcd" in the concatenated string.

The string s is not divisible by string t, so return -1.


- idea: check if len(s) is divisible by len(t) or not first
- if so, repeat t len(s)//len(t) that many times, to make it the same length as s
- then compare if they are equal

In [265]:
def find_repeat(s):
    n = len(s)
    for i in range(1, n+1):
        if n % i == 0 and s[:i] * (n//i) == s:
            return s[:i]

# bad: use loop
# def find_smallest_divisor(s, t):
#     n = len(t)
#     test_string = t
#     i = 1
#     while len(test_string) <= len(s):
#         if s == test_string:
#             return i
#         else:
#             i+=1
#             test_string = t*i
#     return -1

def find_smallest_divisor(s, t):
    len_s = len(s)
    len_t = len(t)
    if len_s % len_t == 0 and t*(len_s//len_t) == s:
        print(len_s//len_t)
        return find_repeat(t)
    else:
        return False 
    

s = 'bcdbcdbcdbcd'
t = 'bcdbcd'

find_smallest_divisor(s, t)

# find_repeat(t)
# find_repeat(s)


2


'bcd'

## hackerrank: Moving Zeros to start/end
- important: moving to start is more difficult than moving to end!
- think 3,1,2,0,3
	- if we swap 3 and 0, then the ordering is changed

Given a list of integers, moving all the zeros to the beginning of the list while keeping the order of all the non-zeros elements unchanged.

 

Requirement:

- In-place implementation (Implement your solution over the original array, don't create new memory to solve this).
- Time Complexity：O(n)

 

Notes:

- Languages: Please use your preferred coding language.

 

Custom Test:

If you would like to define your custom input in the test, please follow the instruction below:

- Saying your input is [1, 2, 3, 0]

- When you define your input in the input box, the first line should be the number of items in your list

- Your input list start from the second line, below is an example:



 

Example:

- [0, 2, 3, 0, 0, 1, 0, 5]  ====> [0, 0, 0, 0, 2, 3, 1, 5]

- [1, 0, 0, 2, 0, 3] ===> [0, 0, 0, 1, 2, 3]

- [0, 0, 0, 0, 0] ==> [0, 0, 0, 0, 0]

- [1, 2, 4, 3] ===> [1, 2, 4, 3]

- [0, 2, 3, 0, 0, 1, 0, 5, 0] ====>[0, 0, 0, 0, 0, 2, 3, 1, 5]

In [282]:
def move_zeros(a):
    # easy way:
    # b = np.array(a)
    # a[a!=0]
    # print([0]*a.count(0) + list(b[b!=0]))
    # [0]*a.count(0) + a[a!=0]
    cnt = 0
    for i, x in enumerate(a):
        if x == 0:
            cnt +=1
        else:
            a[i-cnt] = x
    print(a)

a = [0, 2, 3, 0, 0, 1, 0, 5]
move_zeros(a)

[2, 3, 1, 5, 0, 1, 0, 5]


## moving zero to the end
- idea: two points. left pointer points to the first nonzero entry, and let right pointer match forward. Swap left and right pointers if right points to nonzero entry

In [375]:
def move_zeros_end(a):
    # M1: bad. Ensure one pointer always point to a zero location
    # zero_loc = -1
    # for i, x in enumerate(a):
    #     if x == 0 and zero_loc == -1:
    #         zero_loc = i
    #     elif x != 0 and zero_loc != -1:
    #         a[i], a[zero_loc] = a[zero_loc], a[i]
    #         zero_loc += 1
    # return a

    # M2: let left and right pointer points to the same location at the beginning
    l = r = 0
    for x in a:
        if x != 0:
            a[l], a[r] = a[r], a[l]
            l += 1
        r += 1
    return a 

# a=[0,1,0,3,9]
# a=[0,1,0,2]
# a = [0,1,0,2,1]
a = [0,1,2,0,3]
move_zeros_end(a)

[1, 2, 3, 0, 0]

## hourglasses problem
- NOTE: the input is python list of lists, it's difficult to get a submatrix with it!
	- if really want, need to do list comprehension...
- so, we can just find the value using indexing!

In [316]:
# M1: without numpy
def cal_sum(a, i, j):
    total = 0
    for row in range(i, i+3):
        for col in range(j, j+3):
            if not (row == i+1 and (col == j or col == j+2)):
                total += a[row][col]
    return total

def hourglass_sum(a):
    return max(cal_sum(a, i, j) for i in range(4) for j in range(4))

# M2: with numpy
# def hourglass_sum(a):
#     a = np.array(a)
#     return max(
#         a[i:i+3, j:j+3].sum() - a[i+1,j+0] - a[i+1, j+2] 
#         for i in range(4) 
#         for j in range(4)
#         )

arr=[
[-9, -9, -9,  1, 1, 1],
[ 0, -9,  0,  4, 3, 2],
[-9, -9, -9,  1, 2, 3],
[ 0,  0,  8,  6, 6, 0],
[ 0,  0,  0, -2, 0, 0],
[ 0,  0,  1,  2, 4, 0],
]

# print(arr)

# cal_sum(arr, 1,1)
hourglass_sum(arr)

28

## hackerrank: encryption
- https://www.hackerrank.com/challenges/encryption/problem

In [345]:
def encryption(s):
    s = s.replace(' ', '')
    n = len(s)
    print(n)
    # temp = n**0.5
    col = math.ceil(n**0.5)
    # if isinstance(temp, int):
    #     row = col = temp 
    # else:
    #     row, col = int(temp), int(temp)+1
    # print(row, col)
    res = []
    for i in range(0, len(s), col):
        res.append(s[i:col+i])
        # res.append('\n')
        # print(s[i:col+i])
    # print(res)
    return '\n'.join(res)

# s= 'hello world! I love you so much!!'
# s='if man was meant to stay on the ground god would have given us roots'
s = 'hello'
encryption(s)

5


'hel\nlo'

In [339]:
def encryption(s):
    s = s.replace(' ','') # remove space
    n = len(s) # string length after removing space
    print(n)
    # rows = int(n**0.5)
    n_columns = math.ceil(n**0.5)
    
    i = 0
    res=[]
    while i*n_columns <= n:
        res.append(s[i*n_columns:(i+1)*n_columns])
        i+=1
    

    output = []
    for i in range(n_columns):
        for row in res:
            try:
                output.append(row[i])
            except:
                pass
        output.append(' ')

    return ''.join(output[:-1])

s= 'hello world! I love you so much!!'
encryption(s)

27


'holuh eros! llvo! ldem o!yu wIoc'

# interval problem

## disjoint interval
- given two intervals `(x1,y1)` and `(x2,y2)`, write a function called `is_overlap` that returns True if the two intervals overlap

In [66]:
def is_overlap_helper(i1, i2):
    s1, e1 = i1
    s2, e2 = i2
    # return i1[0] > i2[1] or i2[0] > i1[1]
    return not (s2 > e1 or s1 > e2)

def overlap(intervals):
    # intervals: list of 2-tuples
    # for i1 in intervals:
    count = 0 
    for i in range(len(intervals)):
        for j in range(i+1, len(intervals)):
            # print(i, j)
            count += is_overlap_helper(intervals[i], intervals[j])
            
    return count

i1 = [3,4]
i2 = [5, 6]
is_overlap_helper(i1, i2)
is_overlap_helper(i1, [4,5])
overlap((i1, [4,5], [4,6]))

False

True

3

# recursion

## Fibonacci sequence
- f(0) = 0, f(1) = 1

In [140]:
def fib(n):
    # naive
    # if n<=1: return n
    # return fib(n-2)+fib(n-1)

    # M: dynamic programming (aka bottom-up method). Best method. Also, we can retrieve all fib(n), for n=0,1,...
    # res=[-1]*(n+1)
    # for i in range(0,2): res[i]=i # base case
    # for i in range(2, n+1):
    #     res[i]=res[i-2]+res[i-1]
    # return res, res[n]
    
    # memoization
    memo = [-1]*(n+1)
    def fib_helper(n):
        if memo[n] == -1: # not yet calculated
            memo[n] = fib_helper(n-2) + fib_helper(n-1)
        return memo[n]

@cache
def fib_cache(n):
    return n if n <= 1 else fib_cache(n-1) + fib_cache(n-2)
    
# fibon(10)
fib_cache(10)
fib_cache.cache_info()


55

CacheInfo(hits=8, misses=11, maxsize=None, currsize=11)

## tower of Hanoi
- given three poles. Move all disks in one pole to another. 
- Begin with n disks on peg 1. We can transfer the top n − 1 disks, following the rules of the puzzle, to peg 3 using Hn−1 moves (see Figure 3 for an illustration of the pegs and disks at this point).We keep the largest disk fixed during these moves. Then, we use one move to transfer the largest disk to the second peg. Finally, we transfer the n − 1 disks on peg 3 to peg 2 using Hn−1 moves, placing them on top of the largest disk, which always stays fixed on the bottom of peg 2. This shows that we can solve the Tower of Hano puzzle for n disks using 2Hn−1 + 1 moves.

In [ ]:
def move(n, source, target, auxiliary):
    # move (top) n disks from source to target, using auxiliary
    # return the number of steps needed. In fact, we need 2^n -1 steps
    if n<=0: return 0
    # Move the top n - 1 disks from source to auxiliary, so they are out of the way
    step_1=move(n - 1, source, auxiliary, target)

    # Move the bottom (the bottom one has index n) disk from source to target
    # print(f"move disk {n} from {source} to {target}")
    target.append(source.pop()) # before pop, source only has the bottom one remaining

    # Display our progress
    print(f'A = {A}', f'B = {B}', f'C = {C}', '##############', sep='\n')

    # Move the n - 1 disks that we left on auxiliary onto target
    step_2=move(n - 1, auxiliary, target, source)

    return step_1+1+step_2


def move_count(n, source, target, auxiliary):
    # count steps only
    if n<=0: return 0
    return move_count(n - 1, source, auxiliary, target) + 1 + move_count(n - 1, auxiliary, target, source)


# Initiate call from source A to target C with auxiliary B
n=3
A=list(range(n,0, -1)) # A[0] is the bottom, A[-1] is the top
B=[]
C=[]
move(n, A, C, B)
# move_count(n, A, C, B)

## binary search

In [30]:
def binary_search(a, key):
    def binary_search_auxiliary(a, key, low, high):
        """return the index of `key` if found; return "Not Found" otherwise
            assumes the sequence `a` is sorted.
            Search for a[lo] to a[hi] inclusive. 

        Args:
            a (sequence): sequence of numbers
            key (int/float): key to search for
            low (int): lower index
            high (int): upper index

        Returns:
            int/string: the index of `key`, or `Not Found`
        """    
        if low > high or not a: # not a to deal with empty a 
            return 'NOT FOUND'
        mid = (low + high)//2
        if key < a[mid]:
            return binary_search_auxiliary(a, key, low, mid-1)
        elif key == a[mid]:
            return mid
        else:
            return binary_search_auxiliary(a, key, mid+1, high)
    return binary_search_auxiliary(a, key, 0, len(a)-1)

def binary_search_iterative(a, key):
    low, high = 0, len(a)-1
    while low <= high:
        mid = (low + high)//2
        if key < a[mid]:
            high = mid - 1
        elif key == a[mid]:
            return mid
        else: 
            low = mid + 1
    return "NOT FOUND"

a=[1,2,3,4,5,6,7,8,9,10]
for fun in [binary_search, binary_search_iterative]:
    print(f"using {fun.__name__}...")
    print(fun(a, 10))
    print(fun(a, 12))
    print(fun(a, -1))
    print(fun([7], 7))
    print(fun([7], 0))

using binary_search...
9
NOT FOUND
NOT FOUND
0
NOT FOUND
using binary_search_iterative...
9
NOT FOUND
NOT FOUND
0
NOT FOUND


## tower of Hanoi

## sum by recursion

In [49]:
def sum_rec(a,n):
    """ n is the number of elements in the array"""
    if n == 0:
        return 0
    return a[n-1] + sum_rec(a, n-1) # do not return a[-1]! it will always return the last element of the array! 
sum_rec([1,2,3,4],4)


def sum_rec_binary(a, low, high):
    # use a binary-search-like algorithm
    # seems there is no way to turn it into iterative
    # assume input a nonempty
    if low > high:
        return 0 
    mid = (low+high)//2
    return sum_rec_binary(a, low, mid-1) + a[mid] + sum_rec_binary(a, mid+1, high)
    
print(sum_rec_binary([1,2,3,4],0,3))
print(sum_rec_binary([1,2,3],0,2))
print(sum_rec_binary([1,2],0,1))
print(sum_rec_binary([1],0,0))

10
6
3
1


## calculate pow(a,n) by recursion, assuming n is integer
fast way to compute power: `a^n = (a^n//2)**2` if `n` is even 
- 10^9 = (10^4)**2 * 10
    - so, when `n` is odd, we do `a**(n//2)**2 * a`
- 10^8 = (10^4)**2 
    - so, when `n` is even, we do `a**(n//2)`

In [20]:
def power(a, n: int):
    if n == 0:
        return 1 
    return a * power(a, n-1)
    
# print(power(10,0))
# print(power(10,1))
# print(power(10,2))
# print(power(10,3))

def power_rec_fast(a,n):
    if n == 0:
        return 1

    # M1
    # return (power_rec_fast(a, n//2)**2)*(a**(n%2))

    # M2
    return power_rec_fast(a, n//2)**2 *(1 if n%2==0 else a)
    
print(power_rec_fast(10,0))
print(power_rec_fast(10,1))
print(power_rec_fast(10,2))
print(power_rec_fast(10,3))

1
10
100
1000


## reverse an array in-place
- important: we should use two pointers to control left and right, rather than one, to simplify our code!
- note that we have `reversed` function for sequence, but it's non-inplace!

In [37]:
def reverse_array(a, low, high):
    # inplace
    if low > high:
        return 
    a[low], a[high] = a[high], a[low]
    return reverse_array(a, low+1, high-1)

    # non-inplace too!
    # only need one base case
    # if low > high:
    #     return []
    # return [a[high]] + reverse(a, low, high-1)

    # need two base cases...
    # if low == high:
    #     return [a[high]]
    # elif low > high:
    #     return []
    # return [a[high]] + reverse(a, low+1, high-1) + [a[low]]

def reverse_iterative(a):
    # M1: consider mid-point. More complicated, because using one pointer only, and so need to calculate index directly!
    mid=len(a)//2
    for i in range(mid):
        # for array index i, its counterpart counting from tail is `-i-1` or `len(n)-1-i`
        a[i],a[-i-1]=a[-i-1],a[i]
        # a[i],a[n-1-i]=a[n-1-i],a[i]

    # M2: Use two points. Much simpler! Good!
    i, j = 0, len(a)-1
    while i <= j:
        a[i], a[j] = a[j], a[i]
        i+=1
        j-=1
    
    
# a=[1,2,3,4,5,6]
a=[1,2,3]
# a=[1,2]
reverse_iterative(a)
a
# reverse(a, 0, len(a)-1)

a

[3, 2, 1]

[3, 2, 1]

# number theory

## check if a is a power of b
- check if a=b^k, for some integer k 
- for this problem, we have two choices: 
    - start from b, and multiply itself to get close to a
    - start from a, and divide it by b 
- for the second method, it's faster, since we don't need to wait until b grows to a before we can reject it. 
-  EX: check if a is a power of 2 => check if a=2^k

In [37]:
def is_power(a, b):
    # check if a is a power of b, i.e., a=b^k, for some integer k 
    # return k if it is. return -1 if not
    # a/b => remove a factor of b from a
    # a%b => check if a contains b as a factor 
    count = 0
    while a>1:
        if a%b!=0: return False # b is not a factor of a
        a/=b
        count+=1
    return count

    # M2: bad. need to wait until b is larger than a...
    # check if a = b^k
    idx = 0
    while b**idx <= a:
        if b**idx == a:
            return idx
        idx+=1
    return False 
is_power(4, 2)

2

## count the number of times a given digit appears in a given integer

In [110]:
def count_digit(x, digit=0):
    """
    count the number of times the digit appears in a number.
    Note: if base case is num==0, then we can't deal with count_digit(0,0)!
    """
    
    # M1: recursion
    # this base case can't deal with count_digit(0,0)!
    # if x==0: return 0
    
    # if x<0: x=-x
    # # this base case can deal with count_digit(0,0)!
    # if x<10: return int(x==digit)
    # return count_digit(x//10, digit) + (x%10==digit)

    # M2:
    # if x == 0: return int(x == digit)
    # res = 0
    # while x:
    #     res+= (x%10==digit)
    #     x//=10
    # return res

    # M3
    return sum(1 for d in str(x) if int(d) == digit)

    # M4: use str.count()
    # return str(x).count(str(digit))
    

print(count_digit(123, 4))
print(count_digit(123333, 3))
print(count_digit(0, 0))

0
4
1


## print all perfect n from a to b, inclusive, where n can be any integer 

In [48]:
def print_perfect_n(a, b, n=3):
    # M1: use int(x) == x. might experience round-off error!
    # return [x for x in range(a, b+1) if int(temp := x**(1/n)) == temp]

    # M2: use float().is_integer() method
    # return [x for x in range(a, b+1) if pow(x, 1/n).is_integer()]

    # M3: recursion
    if a>b:
        return []
    return ([a] if pow(a,1/n).is_integer() else []) + print_perfect_n(a+1,b,n)

print_perfect_n(1,100,2)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

## return the largest digit

In [71]:
def largest_digit(x):
    # M0
    # largest = 0
    # while x:
    #     if (digit:=x%10) > largest:
    #         largest = digit
    #     x//=10
    # return largest
    
    # M1: best. find the digits as a list first. then find maximum
    # return max(map(int, str(x)))

    # M1.5: list comprehension
    return max(int(x) for x in str(x))

    # M2: recursion
    # return 0 if x == 0 else max(x%10, largest_digit(x//10))

    # M3: recursion without max function; compatible with C++
    # return (
    #     0 if x == 0 else 
    #     temp if (temp:=largest_digit(x//10)) > (last_digit:=x%10) else 
    #     last_digit
    # )

largest_digit(12345)

5

## sum of digit to power
<!-- find d1^1 + d2^2 + d3^3 +...+dk^k, where n = (d1d2d3...dk) -->
find d1^1 + d2^2 +... dk^k, where n = (dk...d1)

In [83]:
def sum_digit_to_power(n):
    # i, res = 1, 0 
    # while n:
    #     res += (n%10)**i
    #     i+=1
    #     n//=10
    # return res 
    
    # M2
    n_digits = len(str(n))
    res = 0 
    for i in range(1, n_digits+1):
        res += (n%10)**i
        n//=10
    return res 

sum_digit_to_power(1234)

22

## find gcd
- `gcd(a, b) = gcd(b, a%b)` => need to have b > a to take effect 
- NOTE: `if a > b, then gcd(a,b) = gcd(b, b%a) = gcd(b,a)`!
- gcd(a, 0) = a

In [28]:
def gcd(a,b):
    # naive method
    # cur = 1
    # for i in range(2, min(a,b)+1):
    #     if a % i == b % i == 0:
    #         cur = i
    # return cur 
    
    # print(a, b)
    # works even for a > b!
    
    # we need to check `b` because after one function call, b will decrease!
    # if gcd(0, 12) => gcd(12, 0) => 12
    if b == 0: # no need for a == 0!
        return a
    return gcd(b, a%b) # need a >= b to take effect

gcd(3,5)

1

## sum of odd digits - sum of even digits
compute the difference between the sum of values of all odd digits and the sum of values of all even digits. Assume that input x is nonnegative integer

In [7]:
def diff_odd_and_even(x):
    # M1: calculate sum_odd and sum_even first. Bad, since we don't need to know their respective sum in order to solve this problem
    # sum_odd = 0
    # sum_even = 0
    # while x:
    #     digit = x%10
    #     if digit%2 == 0:
    #         sum_even += digit
    #     else:
    #         sum_odd += digit
    #     x //= 10
    # return sum_odd, sum_even

    # M2: recursion
    if x == 0:
        return 0
    digit = x%10
    return diff_odd_and_even(x//10) + (digit if digit%2!=0 else -digit)

    # M3: while loop
    # res = 0
    # while x:
    #     digit = x%10
    #     res += (-digit if digit%2==0 else digit)
    #     x//=10
    # return res

    # M4: recursive one-liner
    # return 0 if x==0 else diff_odd_and_even(x//10) + (last_digit if (last_digit:=x%10)%2 else -last_digit )

x = 12345
diff_odd_and_even(x)


3

## prime number problems
- a number $n\ge 2$ is prime if n is divisible by 1 and itself only
- to do the checking, we can check first all numbers x for $x\le \sqrt{n}$, and if none are factor of $n$, then $n$ is prime   
    - since otherwise, if $n$ only has factor $\gt \sqrt{n}$, then the product of the two prime $ > n$

In [119]:
def is_prime(n: int):
    # M1: by def
    # return all(n%i !=0 for i in range(2, n-2))
    
    # M2: by shortcut
    # if n in [2,3]: return True
    return all(n%i !=0 for i in range(2, floor(n**0.5)+1))
    

def exist_prime(t: list):
    return any(map(is_prime, t)) # use maps
    # return any(is_prime(x) for x in t) # use list comprehension


def list_of_prime(n):
    return [i for i in range(2, n+1) if is_prime(i)]

def return_prime(t: list):
    return [x for x in t if is_prime(x)]

list(map(is_prime, [5,2,1,10, 13, 77, 4])) 
print(is_prime(5),is_prime(2),is_prime(1)) # True True True
print(exist_prime([10,12,14])) # False
print(return_prime([10,12,14,3,7])) # [3,7]

[True, True, True, False, True, False, False]

True True True
False
[3, 7]


## prime factorization

In [120]:
list_of_prime(12)

# is_prime(2)
# n = 2
# ceil(n**0.5)+1

[2, 3, 5, 7, 11]

In [129]:
def prime_factorize(n):
    factors = []
    all_primes = list_of_prime(n)
    i = 0
    while n!=1:
        prime = all_primes[i]
        # print(prime)
        while n%prime == 0:
            factors.append(prime)
            n//=prime
        i+=1
    return factors

prime_factorize(999)
    

[3, 3, 3, 37]

## find all factors of an integer
notice that if q is a factor of n, then n=qk, and k = n/q. So, when we have found the factor q, we can find another factor k too! Since when we write n=qk, either q or k must be less than or equal to sqrt(n), we can just use a for loop from 1 to sqrt(n)

In [273]:
def factors(n):
    # Important: we need to take care of the case when n is perfect square! Then n = a**2 => added `a` twice to our result!
    
    # M1: use set
    # res = set()
    # for i in range(1, int(n**0.5)+1):
    #     if n % i == 0: # i is a factor
    #         res = res.union([i, n//i]) # can use n/i, but this will return a float, not an int!
    # return sorted(res)

    # M2: use list
    res=[]
    for i in range(1,int(n**0.5)+1):
        if n%i==0: # in each step, return two factors, because n = i*q, where q=n//i. Be careful: if n is perfect square, we have the pair (i,i)! 
            res.append(i)
            if n//i != i: res.append(n//i) # the if case is to deal with perfect squares
    return sorted(res)
    
def factors_gen(n):
  for i in range(1,int(n**0.5)+1):
    if n%i==0:
      yield i
      if n//i != i: yield n//i # the if case is to deal with factors(1)
      
print(factors(9)) # [1, 3, 9]
factors(1234)

[1, 3, 9]


[1, 2, 617, 1234]

## number base conversion

In [99]:
def convert_to_binary(n):
    # if n==0: return ''
    # return convert_to_binary(n//2) + str(n%2)
    if n<=1: return str(n)
    return convert_to_binary(n//2) + str(n%2)

def convert_to_hex(n):
    mapping = '0123456789ABCDEF'
    # print(len(mapping))
    if n<=15: return mapping[n]
    return convert_to_hex(n//16) + mapping[n%16]

def convert_to_10(n, a):
    pass 

convert_to_binary(4)
convert_to_hex(123)
    
    

'7B'

In [162]:
def convert_to_10(n, a):
    # `a` is the base that `n` is in
    # e.g., n = 110011, a = 2, then => 51
    assert n%10<=a-1 and n>=0
    if n==0: return 0
    return convert_to_10(n//10, a)*a+ n%10

def ten_to_other_base(n,a):
    # convert to base a
    if n==0: return ''
    return int(str(ten_to_other_base(n//a, a)) + str(n%a)) # we need int because for base case, it expects n to be int!
    # print(n%a, end='')

def convert(n,a,b):
    return ten_to_other_base(convert_to_10(n, a), b)

convert_to_10(111101,2)
ten_to_other_base(1121, 2)
convert(1101101, 2, 8)

61

10001100001

155

## digit distribution

In [90]:
def digit_dist(x):
    # M1: convert to string, then use Counter
    # return Counter(str(x))
    
    # M2: this method will show digits with 0 frequency!
    # res = [0]*10
    # while x:
    #     res[x%10]+=1
    #     x//=10
    # return dict(zip(range(10), res)) 
    
    # M3: traditional
    res = {}
    while x:
        res[last] = res.get(last:= x%10, 0) + 1
        x//=10
    return res

digit_dist(12345)

{5: 1, 4: 1, 3: 1, 2: 1, 1: 1}

## count number of unique digits in an integer

In [21]:
def count_unique_digits(n):
    # M1: convert to string
    # return len(set(str(n)))
    
    # M2: use set
    # deal with base case where n = 0 
    # remember, if n=0, we should return 1, instead of 0!
    # res = set() if n != 0 else {0}
    # while n:
    #     res.add(n%10)
    #     n//=10
    # return len(res)

    # M3: use list, and check whether a digit is already in the list before append
    res = [] if n != 0 else [0]
    while n:
        if (d:=n%10) not in res:
            res.append(d)
        n//=10
    return len(res)

n = 123112
# n = 0
count_unique_digits(n)
count_unique_digits(123451122131) # 5

3

5

## number of digits

In [207]:
def n_digits(x):
    # M1: recursion
    # if x<0: return n_digits(-x)
    # if x < 10: return 1
    # return 1 + n_digits(x//10)

    # M2: log.
    # log(10^3) = 3 => log(x) = 3, if 10^3<=x<10^4 => x is a 4 digit number
    return int(np.log10(abs(x))+1) if x !=0 else 1 # or, int(np.log10(abs(x)))+1
    
    # M3: while loop
    # ans = 1
    # x = abs(x)
    # while x > 9:
    #     ans += 1
    #     x//=10
    # return ans
    
    # M4: convert to string first
    return len(str(abs(x)))

for x in [12345, 100, 10, 1, 0]:
    print(f"n_digits({x}) = {n_digits(x)}")

n_digits(12345) = 5
n_digits(100) = 3
n_digits(10) = 2
n_digits(1) = 1
n_digits(0) = 1


## find nth digit
- Note that the rightmost digit is defined to be the first digit
- def: we are treating the rightmost digit as the first digit.
- logic: we know `n%10` => 1st digit. So, n//10%10 => 2th digit. So, in general, `n//10**(i-1)%10 to get ith digit`

In [69]:
def nth_digit(n, i):
    # recursion
    # if i == 1:
    #     return n%10
    # return nth_digit(n//10, i-1)

    # while loop
    # while i != 1:
    #     n //= 10
    #     i -= 1
    # return n%10

    # convert to string
    # return int(str(n)[-i])

    # direct formula. fastest
    # the rightmost digit is with index 1
    # think: 
        # n%10 get 1st digit
        # (n//10)%10 get 2nd digit
        # (n//10**2)%10 get 3nd digit
    return (n // (10**(i-1))) % 10

x = 123456
for i in range(1, n_digits(x)+1):
    nth_digit(x, i)

6

5

4

3

2

1

## remove leftmost digit

In [ ]:
def remove_leftmost_digit(n):
    # this strategy can't generalize to "remove k_th digit"
    # EG: 1234 => 1234-200= 1034 =/= 134
    n_digits=int(np.log10(n))+1
    leftmost_digit = n//10**(n_digits-1)%10
    return n - leftmost_digit*10**(n_digits-1)

remove_leftmost_digit(9234) # 234
remove_leftmost_digit(1234) # 234

## remove nth digit

In [79]:
def remove_nth_digit(n, i):
    # M1: convert to string, then using slicing to exclude the given digit index
    # s = str(n)
    # i_reverse = len(s) - i 
    # return s[:i_reverse] + s[i_reverse+1:]

    # M2: convert to string, then to list to use pop methdod
    res = list(str(n))
    res.pop(-i)
    return int(''.join(res))

remove_nth_digit(1234, 2)

124

## Armstrong number
- 1634 is Armstrong, since `1634=1^4+6^4+3^4+4^4`, where `4` is # digits
- in `res+=(n%10)**n_digits`, remember the parenthesis is necessary!!

In [133]:
def is_armstrong(n):
    # M1: standard
    # n_cpy = n
    # n_digits = len(str(n))
    # res = 0
    # while n:
    #     res += (n%10)**n_digits
    #     n//=10
    # return res == n_cpy
    
    # M2: convert the digits as a list first
    # digits = np.array([int(digit) for digit in str(n)])
    # return np.sum(digits**len(digits)) == n 

    # M3: without np
    digits = [int(digit) for digit in str(n)]
    return sum(digit**len(digits) for digit in digits) == n

def armstrong(lower=0,upper=100):
    if lower>upper: lower,upper=upper,lower
    return [i for i in range(lower, upper+1) if is_armstrong(i)]

# is_armstrong(1634)
armstrong(1, 10000)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 153, 370, 371, 407, 1634, 8208, 9474]

## reverse of an integer
- note that we don't need to use power! we can multiply the result by 10 each time we iterate!
- this is different from reversing a string, since the input is integer!

- M1 (bad):  
`12345 = 5*10^0 + 4*10^1 + 3*10^2 + 2*10^3 + 1*10^4`  
`54321 = 1*10^0 + ....`

- M2 (good):
`res = res*10 + x%10`

In [6]:
def reverse_num(x):
    n_digits = int(np.log10(abs(x))) + 1 if x !=0 else 1
    res = 0
    
    # M1: bad. use complicated method to find power index 
    # for i in range(1, n_digits+1):
    #     res += (x%10) * (10**(n_digits-i))
    #     x //= 10 
    
    # M2: choose a more understandable range to iterate
    # for i in range(n_digits-1, -1, -1):
    #     res += (x%10) * (10**i)
    #     x //= 10 
    
    # M3: improved M2, with while. We should remember this strategy by heart! very often we don't need to know the number of digits of a number to begin with!!!
    # while x != 0:
    #     res = res*10 + x%10
    #     x //= 10
    # return res
    
    # M4: change to string first, then get the reverse by [::-1], and finally change the string to int by int() constructor 
    return int(str(x)[::-1])

reverse_num(1234)
# reverse_num(1)
# reverse_num(12)
# reverse_num(3)

4321

# dynamic programming

## dynamic programming to compute probability mass function
<!-- - $P(X=k+1) = \frac{p}{1-p}\frac{n-k}{k+1}P(X=k)$ -->
- $P_k = \frac{p}{1-p}\frac{n-k+1}{k}P_{k-1}$, for $X\sim B(np)$
- $P_k = \frac{\lambda}{k}P_{k-1}$, for $X\sim poi(\lambda)$
- $P_k = qP_{k-1}$, for $X\sim geo(p)$
- $P_k = \frac{k-1}{k-r}qP_{k-1}$, for $X\sim \text{negbinom}(p, r)$

In [ ]:
# --------------------------------- binomial --------------------------------- #
n = 30
p = 0.3
q = 1-p
prob = [0]*(n+1)
prob[0] = q**n
for i in range(1, n+1):
    # prob[i] = p/q * (n-(i-1))/((i-1)+1) * prob[i-1]
    prob[i] = p/q * (n-i+1)/i * prob[i-1]
# prob    
sum(prob)

# ---------------------------------- poisson --------------------------------- #
n = 10
lam = 3
prob = [0]*(n+1)
prob[0] = np.exp(-lam)
for i in range(1, n+1):
    prob[i] = lam/i * prob[i-1]
# prob    
sum(prob)

# --------------------------------- geometric -------------------------------- #
n = 50
p = 0.3
prob = [0]*(n+1)
prob[1] = 0.3
for i in range(2, n+1):
    prob[i] = (1-p)* prob[i-1]
sum(prob)

# ---------------------------- negative binominal ---------------------------- #
n = 60
p = 0.3
r = 5
prob = [0]*(n+1)
prob[r] = p**r
for i in range(r+1, n+1):
    prob[i] = (i-1)/(i-r)*(1-p)* prob[i-1]
sum(prob)

0.9999999999999988

0.9997076630493527

0.9999999820153496

0.9999900954301846

## binomial coefficient calculation
- if we use the formula $\binom{n}{r} = \binom{n-1}{r-1} + \binom{n-1}{r}$, then we have overlapping subproblems, and so need to use dynamic programming
- if we use the formula $\binom{n}{r} = \frac{n}{r} \binom{n-1}{r-1}$, then there is no overlapping subproblems!

In [2]:
@cache
def binomial(n,r):
    if r == 0 or r == n:
        return 1
    assert n >= r
    return binomial(n-1, r-1) + binomial(n-1, r)
    
    # in fact, no need to use dynamic programming, since we have another formula
    # return n/r* binomial(n-1, r-1)

binomial(1000, 30)
binomial.cache_info()

2429608192173745103270389838576750719302222606198631438800

CacheInfo(hits=28101, misses=30100, maxsize=None, currsize=30100)

## rod cutting

In [31]:
@cache
def rod_cut(price):
    # print(price)
    
    n = len(price)
    
    # ------------------------ this base case not needed! ------------------------ #
    # if (n:=len(price)) == 1:
    #     return price[0]
    # ---------------------------------------------------------------------------- #
    if n == 0:
        return 0 
    
    return max(price[i] + rod_cut(price[:-(i+1)]) for i in range(0, n))

    # ----------------- another version that requires n as input ----------------- #
    # print(price)
    # if n == 0:
    #     return 0
    
    # return max(price[i] + rod_cut(n-(i+1), price) for i in range(0, n))
# ---------------------------------------------------------------------------- #

# price = np.random.random(1000)
price = [1,5,8,9,10,17,17,20,24,30]
# price = [1]

rod_cut(tuple(price))
rod_cut.cache_info()
    
    

30

CacheInfo(hits=45, misses=11, maxsize=None, currsize=11)

## change making
- idea: res[i,j] = using coins[:i], minimum # coins summing to j
    - optimal solution either use coins[i-1] or not
- `res[i,j] = min(res[i-1, j], res[i, j-w[i-1]]+1)`

In [19]:
@cache
def change_making(c,w):
    # print(c, w)
    if len(c) == 0 and w != 0:
        return np.inf
    elif w == 0:
        return 0
    
    if w >= c[-1]:
        return min(
            change_making(c[:-1], w),
            change_making(c, w-c[-1]) + 1
            )
    else:
        return change_making(c[:-1], w)

# c=[6,4,1,3,2,7]
c=np.random.randint(1, 100, 1000)
# c=[6,4,1]
w=100
change_making(tuple(c), w)
change_making.cache_info()

2

CacheInfo(hits=49963, misses=99099, maxsize=None, currsize=99099)

## matrix chain
- d[0,1] = matrix size of matrix 0
- d[1,2] = matrix size of matrix 1
- d[i,i+1] = matrix size of matrix i 

In [ ]:
def matrix_chain(d):
    min(matrix_chain(d[:k]) + matrix_chain(d[k+1:]) + d[0] for k in range(0, len(d)))
    

## maximum subarray

In [66]:
def max_subarray(a):
    # if len(a) == 1:
    #     return max(0, a[0])
    # return max(max_subarray(a[:-1]), max_subarray(a[:-1]) + a[-1])
    
    # if len(a) == 0:
    #     return 0 
    # elif len(a) == 1:
    #     return a[0]
    # n = len(a)
    # return max(max(max_subarray(a[:i])+a[i], a[i]) for i in range(n))
    
    # b = [np.inf]*len(a)
    # b[0] = a[0]
    # for i in range(1, len(a)):
    #     b[i] = max(b[i-1] + a[i], a[i])
    # print(b)
    # return max(b)

    cur = a[0]
    for i in range(1, len(a)):
        temp = max(cur + a[i], a[i])
        cur = max(cur, temp)
    return cur
        

# data=[13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5,  -22,  15,  -4,7]
data=[13, -3, -25, 10]
# data=[-1,2,3]
# data=[-1]
# data=[-1,-2,-3]
# data=[1,1,-10,-2,-1]

max_subarray(data)

23

 ## house robber
 - given price for each house
 - can't rob adjacent houses
 - res[i] = optimal val for robbing first i houses
 - consider whether the last house is robbed or not

In [33]:
@cache
def rob(vals):
    # vals[i] = house value of ith house, starting from 0
    if (n:=len(vals))==0:
        return 0
    
    # elif n == 1:
    #     return vals[0]
    
    # if len(vals) <= 1, then vals[:-2] will still give us empty list
    return max(rob(vals[:-1]), vals[-1] + rob(vals[:-2]))

vals = (1,2,3,1)
# vals = np.random.random(size=10**4)
rob(tuple(vals))
rob.cache_info()

4

CacheInfo(hits=4, misses=5, maxsize=None, currsize=5)

## subset sum

## dynamic probability problem

### 688. Knight Probability in Chessboard
- it's like a tree diagram. We're exploring all the possibilities

In [ ]:
# class Solution:
#     def knightProbability(self, n: int, k: int, row: int, column: int) -> float:
#         pass

from itertools import product

def knightProbability(n: int, k: int, row: int, column: int) -> float:
    
    # if row>=n or row < 0 or column >=n or column <0:
    #     return 0 
    # if k==0:
    #     return 1
    
    # combs = list(product([2,-2],[1,-1])) + list(product([1,-1],[2,-2]))
    # # M1
    # res = []
    # for row_val, col_val in combs:
    #     res.append(knightProbability(n, k-1, row+row_val, column+col_val))
    # return sum(res)/len(res)
    
    # M2:
    # return sum(knightProbability(n, k-1, row+row_val, column+col_val) for row_val, col_val in combs)/len(combs)

    # M3: dynamic. memoization
    res = {}
    combs = list(product([2,-2],[1,-1])) + list(product([1,-1],[2,-2]))
    def helper(k, row, column):
        if row>=n or row < 0 or column >=n or column <0:
            return 0 
        if k==0:
            return 1        
        if (k,row,column) not in res:
            res[(k,row,column)] = 1/8*sum(helper(k-1, row-i, column-j) for i, j in combs)
        return res[(k,row,column)]
    return helper(k, row, column)
    

knightProbability(n = 100, k = 20, row = 0, column = 0)
# knightProbability(n = 1, k = 0, row = 0, column = 0)
    

0.022654876508337453

### 808. Soup Servings

In [ ]:
from functools import cache
# sys.setrecursionlimit(100000)
def soupServings(n: int) -> float:
    res = {}
    def helper(n_A, n_B):
        if n_A==0 and n_B!=0:
            return 1
        if n_A==n_B==0:
            return 0.5
        if n_A !=0 and n_B==0:
            return 0 
        if (n_A, n_B) not in res:
            res[(n_A, n_B)] = 1/4*sum(helper(max(0, n_A-i), max(0,n_B-j)) for i,j in [(100,0), (75,25), (50,50), (25, 75)])
        return res[(n_A, n_B)]
    
    # @cache
    # def helper(n_A, n_B):
    #     if n_A==0 and n_B!=0:
    #         return 1
    #     if n_A==n_B==0:
    #         return 0.5
    #     if n_A !=0 and n_B==0:
    #         return 0 
    #     return 1/4*sum(helper(max(0, n_A-i), max(0,n_B-j)) for i,j in [(100,0), (75,25), (50,50), (25, 75)])
    return helper(n, n)
    # pass
    
soupServings(n=4900)

0.9999959599566788

### 837. New 21 Game

In [ ]:
import functools

# @functools.cache
def new21Game(n: int, k: int, maxPts: int) -> float:
    res = {}
    def helper(n, k):
        if k <= 0:
            return 1 if n >= 0 else 0 
        if (n,k) not in res:
            res[(n,k)] = 1/maxPts * sum(helper(n-i, k-i) for i in range(1, maxPts+1))
        return res[(n,k)]
    return helper(n,k)



# new21Game(n = 10, k = 1, maxPts = 10)
# new21Game(n = 100, k = 90, maxPts = 30)
# new21Game(n = 6, k = 1, maxPts = 10)
# new21Game(n = 21, k = 17, maxPts = 10)

0.7327777870686081

### 1230. Toss Strange Coins (Medium)
- let ans[i,j] = P(getting j heads in first i flips)
- $ans[i,j] = ans[i-1, j]*(1-prob[i]) + ans[i-1, j-1]*prob[i]$,

In [ ]:
def toss_coins(prob, target):
    # M1: naive 
    # prob = np.array(prob)
    # res = []
    # for comb in itertools.combinations(range(len(prob)), target):
    #     comb = list(comb)
    #     res.append(prob[list(comb)].prod() * (1-np.delete(prob, comb)).prod())
    # return sum(res)

    # M2: dp
    # np.zeros(shape=(len(prob), ))
    res = {}
    def helper(n, target):
        print(n, target)
        # if target == n == 1:
        #     return prob[0]
        if target == 0:
            # print(1-np.array(prob[:n]))
            return (1-np.array(prob[:n])).prod()
            # return helper(n-1,target)*(1-prob[n-1])
        if target > n:
            return 0
        if (n, target) not in res:
            res[(n, target)] = helper(n-1, target-1) * prob[n-1] + helper(n-1, target) * (1-prob[n-1])
        return res[(n, target)]
    
    return helper( len(prob), target)
    
    
prob = [0.4]*3
# prob = [0.4]*1
target = 2
toss_coins(prob, target)

3 2
2 1
1 0
1 1
0 0
0 1
2 2
1 1
1 2


0.28800000000000003

# stack problem
- rotate sorted array: https://www.youtube.com/watch?v=nIVW4P8b1VA&list=PL4BNnTqnsOLdwk_5UUOv3FN57fMtfBo6c&index=13&t=542s
- schedule CPU tasks: https://www.youtube.com/watch?v=RR1n-d4oYqE&list=PL4BNnTqnsOLdwk_5UUOv3FN57fMtfBo6c&index=5
- Decode String: https://www.youtube.com/watch?v=qB0zZpBJlh8&list=PL4BNnTqnsOLdwk_5UUOv3FN57fMtfBo6c&index=18&t=542s


## 735. Asteroid Collision
We are given an array asteroids of integers representing asteroids in a row.

For each asteroid, the absolute value represents its size, and the sign represents its direction (positive meaning right, negative meaning left). Each asteroid moves at the same speed.

Find out the state of the asteroids after all collisions. If two asteroids meet, the smaller one will explode. If both are the same size, both will explode. Two asteroids moving in the same direction will never meet.

In [ ]:
def collision(asteroids):
    stack = []
    for a in asteroids:
        if len(stack) == 0:
            stack.append(a)
        while stack[-1] > 0 and a < 0:
            if stack[-1] < abs(a):
                stack.pop()
                stack.append(a)
        else:
            
                
            

In [ ]:
class Stack:
    def __init__(self):
        self.data = []
    def push(self, val):
        self.data.append(val)
    def pop(self):
        return self.data.pop()
    def is_empty(self):
        return len(self.data) == 0 
    def peek(self):
        return self.data[-1]
    def get_data(self):
        return self.data


def collision(cars):
    # https://www.youtube.com/watch?v=LN7KjRszjk4&list=PLot-Xpze53lfxD6l5pAGvCD4nPvWKU8Qo&index=7
    # stack = []
    # for c in cars:
    #     push_to_stack = True
    #     while stack and stack[-1]>0 and c<0:
    #         # diff = c + stack[-1]
    #         if stack[-1] < abs(c): # need to destroy stack[-1], and push c
    #             stack.pop()
    #         elif stack[-1] == abs(c): # need to destroy stack[-1], but no need to push c
    #             stack.pop()
    #             push_to_stack = False
    #             break
    #         else: # no need to destroy stack[-1]
    #             push_to_stack = False
    #     if push_to_stack:
    #         stack.append(c)
    #     print(stack)
    # return stack

    # stack = Stack()
    # for c in cars:
    #     push_to_stack = True
    #     while not stack.is_empty() and stack.peek()>0 and c<0:
    #         # diff = c + stack.peek()
    #         if stack.peek() < abs(c): # need to destroy stack.peek(), and push c
    #             stack.pop()
    #         elif stack.peek() == abs(c): # need to destroy stack.peek(), but no need to push c
    #             stack.pop()
    #             push_to_stack = False
    #             break
    #         else: # no need to destroy stack.peek()
    #             push_to_stack = False
    #     if push_to_stack:
    #         stack.push(c)
    # return stack.data

    # simplified version: all weights are distinct
    stack = Stack()
    for c in cars:
        push_to_stack = True
        while not stack.is_empty() and stack.peek()>0 and c<0:
            if stack.peek() < -c: # need to destroy stack.peek()
                stack.pop()
            else: # no need to destroy stack.peek()
                push_to_stack = False
                break
        if push_to_stack:
            stack.push(c)
    return stack.get_data()
    
# cars = [3,4,-1]
# cars = [5,5,-5]
# cars = [3,4,-6]
cars=[1,3,4,-6, -3, -1]
collision(cars)

In [ ]:
# def explode(cars):
#     stack = Stack()
#     for c in cars:
#         push_to_stack = True
#         while not stack.is_empty() and stack.peek()>0 and c<0:
#             if stack.peek() < -c: # need to destroy stack.peek()
#                 stack.pop()
#             else: # no need to destroy stack.peek()
#                 push_to_stack = False
#                 break
#         if push_to_stack:
#             stack.push(c)
#     return stack.get_data()

def explode(cars):
    stack = Stack()
    for c in cars:
        push_to_stack = True
        while not stack.is_empty() and stack.peek()>0 and c<0:
            if stack.peek() < -c:
                stack.pop()
            else:
                push_to_stack= False
                # break
        if push_to_stack:
            stack.push(c)
    return stack.get_data()        

# give 10 marks for this
# def explode(cars) :
#     # wrong: when cars=[-3,4]
#     stack = Stack() # you must use this stack variable
#     # Your code for part (c) should be put here
#     for x in cars[::-1]:
#         if stack.is_empty():
#             stack.push(x)
#         else:
#             if (stack.peek() > 0 and x > 0) or (stack.peek() < 0 and x < 0):
#                 stack.push(x)
#             elif (stack.peek() < 0 and x > 0):
#                 if abs(stack.peek()) < abs(x):
#                     stack.pop()
#                     stack.push(x)
#             elif (stack.peek() > 0 and x < 0):
#                 if abs(stack.peek()) < abs(x):
#                     stack.pop()
#                     stack.push(x)
#     return stack.get_data()[::-1]


explode([1,2,3,-5,-6,-7,10])
# explode([1,2,3,-5])
# explode([10,2,-3])
# explode([4,10,-3])
# explode([4,10,-3])
# explode([-1,-2,-3])
# explode([-4, 1, 2, 3])
# explode([])


## 20. Valid Parentheses
Given a string s containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

An input string is valid if:

Open brackets must be closed by the same type of brackets.
Open brackets must be closed in the correct order.
Every close bracket has a corresponding open bracket of the same type.


- idea: for left brackets, put to stack. For right brackets, check if it matches the top bracket in stack
- important: need to create a string to check whether the close backet matches the corresponding open bracket, by `.index` method
- whenever we use `stack[-1]`, remember to make sure the stack is nonempty!

In [7]:
def is_matched(s):
    stack = []
    left= '([{' #     left = ['(', '[', '{']
    right= ')]}' #     right = [')', ']', '}']
    for c in s:
        if c in left:
            stack.append(c)
        elif c in right:
            if len(stack)==0 or left.index(stack.pop()) != right.index(c):
                return False
    return len(stack)==0

# is_well_formed('({*5*(4+3))}]')
print(f"{is_matched('[(5+x)-(y+z)]') = }") # True
print(f"{is_matched('[') = }") # False
print(f"{is_matched('') = }") # True
print(f"{is_matched('[[') = }") # False
print(f"{is_matched(']') = }") # False
print(f"{is_matched('([{}])') = }") # True
print(f"{is_matched('([{[r]}])') = }") # True

is_matched('[(5+x)-(y+z)]') = True
is_matched('[') = False
is_matched('') = True
is_matched('[[') = False
is_matched(']') = False
is_matched('([{}])') = True
is_matched('([{[r]}])') = True


## postfix evaluation

In [3]:
def postfix_eval(s):
    stack = []
    for c in s:
        if c.isdigit():
            stack.append(int(c))
        # elif c in ['+','-','*','/']:
        elif c in "+-*/":
            assert len(stack) >= 2
            # print(f'{left}{c}{right}')
            # stack.append(eval(f'{left}{c}{right}'))
            stack.append(eval(f'{stack.pop()}{c}{stack.pop()}'))
        else:
            raise TypeError("error character type!")
    assert len(stack) == 1
    return stack[-1]

s='1234*+-'
postfix_eval(s)

13

# ad-hoc technique

## 1093. Statistics from a Large Sample

In [57]:
# def sampleStats(count: List[int]) -> List[float]:
def sampleStats(count: list[int]) -> list[float]:
    # min, max, mean, median, mode
    # min: find the first entry in count being > 0 
    # max: find the last entry in count being > 0 
    # mean: find the total first, then divided by sum(count)
    # median: 
    
    res = {
        'min': float('+inf'),
        'max': float('-inf'),
        'mode': 0,
        'max_freq': 0,
        'sum': 0,
        'n_elements': 0,
        'mean': 0
        }
    for val, cnt in enumerate(count):
        # can use `continue` instead
        if cnt > 0:
            if res['max'] < val:
                res['max'] = val
            if res['min'] > val:
                res['min'] = val
            # res['max'] = max(res['max'], val)
            # res['min'] = min(res['min'], val)
            res['sum'] += val*cnt
            res['n_elements'] += cnt
            
            if cnt > res['max_freq']:
                res['max_freq'] = cnt
                res['mode'] = val
    
    res['mean'] = res['sum'] / res['n_elements']
    return res
    pass


# count = [0,1,3,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
count = [0,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

sampleStats(count)


# from typing import List
# def sampleStats(count: List[int]) -> List[float]:
#     # min, max, mean, median, mode
#     res = [np.inf, -1, -1, -1, -1]
#     total_cnt = 0
#     tot = 0
#     # total_cnt = sum(x for x in enumerate(count))
#     total_cnt = sum(count)
#     cur_cnt = 0
#     median_pos = total_cnt//2+1 if total_cnt%2 == 1 else (total_cnt//2, total_cnt//2+1)
        
#     for i, cnt in enumerate(count):
#         if cnt == 0:
#             continue
#         # total_cnt += cnt
#         cur_cnt += cnt
#         tot += i*cnt
#         if i < res[0]: # update min
#             res[0] = i
#         if i > res[1]: # update max
#             res[1] = i 
#         if cnt > res[-1]: # update mode
#             res[-1] = cnt


# count = [0,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


# sampleStats(count)

{'min': 1,
 'max': 4,
 'mode': 1,
 'max_freq': 5,
 'sum': 9,
 'n_elements': 6,
 'mean': 1.5}

## 1467. Probability of a Two Boxes Having The Same Number of Distinct Balls

In [ ]:
def getProbability(self, balls: List[int]) -> float:
    pass

# backtracking problem

## powerset generation

In [ ]:
# backtracking: https://www.youtube.com/watch?v=REOH22Xwdkk
def powerset(a):
    res = []
    subset = []
    def dfs(i): # consider 
        if i>=len(a): # out of bound 
            res.append(subset.copy())
            return

        subset.append(a[i]) 
        dfs(i+1) # all solutions with a[i]

        print(subset)

        subset.pop()
        dfs(i+1)
    dfs(0)
    return res

a=[1,2,3]
powerset(a)

[1, 2, 3]
[1, 2]
[1, 3]
[1]
[2, 3]
[2]
[3]


[[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]

## parenthesis generation

In [ ]:
def generate_parenthesis(n: int) -> list:
    # To understand
    stack = []
    res = []

    def backtrack(openN, closedN):
        if openN == closedN == n:
            res.append( "".join(stack) )
            # print(res)
            return
        
        if openN < n:
            stack.append("(")
            backtrack(openN+1, closedN)
            stack.pop()

            print(stack)
        
        if closedN < openN:
            stack.append(")")
            backtrack(openN, closedN+1)
            stack.pop()


    backtrack(0,0)
    return res
        
generate_parenthesis(3)
# print("hello")

['(', '(']
['(', '(', ')']
['(', '(', ')', ')']
['(']
['(', ')', '(']
['(', ')', '(', ')']
['(', ')']
[]


['((()))', '(()())', '(())()', '()(())', '()()()']

# hashing

In [ ]:
# data = [2341, 4234, 2839, 22, 397, 3920]
# data = [5, 28, 19, 15, 20, 33, 12, 17, 10]
data = [4,6,20,14,31,29]
n = 5
for x in data:
    print(f'{x} hashes to {x%n}')

4 hashes to 4
6 hashes to 1
20 hashes to 0
14 hashes to 4
31 hashes to 1
29 hashes to 4


In [ ]:
def find_hash(data, n=7):
    table = dict(zip(range(n), ['unfilled']*n  ))
    for x in data:
        counter = 0 
        while counter <n and table[(x+counter)%n]!='unfilled':
            counter+=1
        if counter == n:
            print(f"can't insert {x}")
        else:
            table[(x+counter)%n] = x
    return table

data = [2341, 4234, 2839, 22, 397, 3920]
find_hash(data)


{0: 3920, 1: 22, 2: 'unfilled', 3: 2341, 4: 2839, 5: 397, 6: 4234}

In [ ]:
data = [2341, 4234, 2839, 22, 397, 3920]
# data = [5, 28, 19, 15, 20, 33, 12, 17, 10]
# data = [10,22,31,4,17]
n = 7   
# table = ['unfilled']*n
table = dict(zip(range(7), ['unfilled']*7  ))
# table
for x in data:
    counter = 0 
    while counter <n and table[(x+counter)%n]!='unfilled':
        counter+=1
    if counter == n:
        print(f"can't insert {x}")
    else:
        table[(x+counter)%n] = x
    # print(table)


    # while pos < n and table[pos] != 'unfilled':
    #     pos+=1
    # if pos < n: 
    #     table[pos] = x
    # else:
    #     print(f"can't insert {x}")  
table

{0: 3920, 1: 22, 2: 'unfilled', 3: 2341, 4: 2839, 5: 397, 6: 4234}

# misc technique

## loop over powerset using built-in itertools.combinations

In [203]:
n = 4
cnt = 0 
for i in range(n+1):
    for comb in combinations(range(n), i):
        cnt+=1
        print(comb)
print(cnt)

# ------------------- for getting subarray from a 2d array ------------------- #
# np.random.seed(2)
# n_samples = 2
# n_coef = 4
# X = pd.DataFrame(np.random.normal(size=(n_samples,n_coef)), columns=[f"x_{i+1}" for i in range(n_coef)])
# y = X.sum(axis=1) + np.random.normal(loc=2, scale=4, size=n_samples)
# X.insert(0, 'const', 1)

# for i in range(1, X.shape[1]):
#     for comb in combinations(range(1, X.shape[1]), i):
#         X.iloc[:, [0] + list(comb)]

()
(0,)
(1,)
(2,)
(3,)
(0, 1)
(0, 2)
(0, 3)
(1, 2)
(1, 3)
(2, 3)
(0, 1, 2)
(0, 1, 3)
(0, 2, 3)
(1, 2, 3)
(0, 1, 2, 3)
16


## find all elements in list b that in list a
- useful for finding no submission
- basically, find b intersection a

In [ ]:
def find_intersect(a,b):
    # M1: list comprehension
    # return [x for x in a if x in b]
    
    # M2: np
    return np.array(a)[np.isin(a,b)] # remember that np.isin() is a static method taking two arrays as input!

a = [1,1,1,2,3,4]
b = [1,2,3,4,5,6,7]
# find_intersect(a,b)
find_intersect(b,a)

array([1, 2, 3, 4])

## find common elements among several lists
- that is, return an element if it appears in ALL lists 
- if we want to find elements that appear in ANY list, a slick strategy is to convert everything to set and then use union
- if we want to find elements that appear in ALL lists, a slick strategy is to convert everything to set and then use intersection
- for anything in between, a good method is to build a frequency distribution first. 

In [84]:
from itertools import chain
def appear_all(*t):
    return set(t[0]).intersection(*t[1:])

def appear_any(*t):
    return set(t[0]).union(*t[1:])

def appear_at_least_k(k,*lists, remove_duplicates_in_list=False):
    if remove_duplicates_in_list:
        lists = [set(lis) for lis in lists]
    
    # # M1: use Counter
    # # MM1: convert to one single lists using `itertools.chain`
    # counter = Counter(chain(*lists))
    
    # # MM2: use list comprehension instead
    # counter = Counter([ele for lis in lists for ele in lis])
    
    # # MM3: use Counter.update
    # counter = Counter()
    # for lis in lists:
    #     counter.update(lis)

    
    # M2: use dict, without Counter
    # counter = dict()
    # for lis in lists:
    #     for ele in lis:
    #         counter[ele] = counter.get(ele, 0) + 1
    
    # return sorted((key, val) for key, val in counter.items() if val>=k)

    # -------------------------------- M3: use np -------------------------------- #
    lists_flatten=np.hstack(lists) # best method
    # lists_flatten=np.hstack([*lists]) # same as np.hstack([lis for lis in lists])
    # lists_flatten=np.hstack([np.unique(lis) for lis in t]) if need to remove duplicates. We cannot use np.hstack(*t) here because we need to eliminate the duplicate values of each list
    unique, counts = np.unique(lists_flatten, return_counts=True)

    index=counts>=k
    # return np.vstack((unique[index], counts[index])).T
    # return index.sum()  # return only the total number of elements with freq>=k. Or, use np.count_nonzero(index) 
    # return [pair for pair in zip(unique, counts) if pair[1]>=k] # return as a list
    return list(zip(unique[index], counts[index])) # return as a list
    # ---------------------------------------------------------------------------- #
    
    # ------------------------------ M4: use pandas ------------------------------ #
    # lists_flatten = np.hstack(lists) # same as np.hstack([lis for lis in lists])
    # return pd.Series(lists_flatten).value_counts()[lambda x: x >= k].to_dict()
    # ---------------------------------------------------------------------------- #
    
    
    


np.random.seed(1)
a1=list(np.random.randint(30, size=10**1))
a2=list(np.random.randint(30, size=10**1))
a3=list(np.random.randint(30, size=10**1))
# appear_all(a1,a2,a3)

# appear_any(a1,a2,a3)
appear_at_least_k(3, a1,a2,a3)

{5: 3, 28: 3, 18: 3, 11: 3}

## special case: find the intersection of three sequences

In [ ]:
def intersection_three(s1,s2,s3):
    # Naive:
    # res=[]
    # for a in set(s1):
    #     for b in s2:
    #         for c in s3:
    #             if a==b==c:
    #                 res.append(a)
    # return res


    # fast method
    res=[]
    # changing s1 to set is necessary. THink about: [1,1,2], [1,1,1], [1,1,1] => 1 will be counted multiple times.
    # we just need to change s1 to set; no need for s2 and s3
    for a in set(s1):
        for b in s2:
            if a==b:
                for c in s3:
                    if a==c:
                        res.append(a)
                        break
                break
    return res

    # method 2. slower
    # res=[]
    # s2_unique=set(s2)
    # s3_unique=set(s3)
    # for x in set(s1):
    #     if x in s2_unique and x in s3_unique:
    #         res.append(a)
    # return res

# s1=[1,2,3,1,1,3]
# s2=[1,1,4,6,4,1,1,3]
# s3=[1,1,3,1,7,8]
s1=np.random.randint(1000,size=10**7)
s2=np.random.randint(1000,size=10**7)
s3=np.random.randint(1000,size=10**7)
%time intersection_three(s1,s2,s3)

## print numbers from a to b, but exclude some given numbers

In [14]:
def print_nums(a, b, exclude_digits = [0,1,3]):
    # version 1: exclude those numbers in exclude_digits
    # print(*[i for i in range(a,b+1) if i not in exclude_digits])
    
    # print all numbers from a to b, inclusive. Exclude those numbers whose last digit is in `exclude`
    
    # return the numbers as list only
    # return [x for x in range(a,b+1) if x%10 not in exclude_digits]
    
    # print 
    # print (*[x for x in range(a,b+1) if x%10 not in exclude_digits], sep=' ')
    
    # recursion
    if a>b: return 
    if a%10 not in exclude_digits:
        print(a, end=' ')
    print_nums(a+1, b, exclude_digits)
    

a = 1
b = 30
print_nums(a, b)

2 4 5 6 7 8 9 12 14 15 16 17 18 19 22 24 25 26 27 28 29 

## generate a list with some excluded elements

In [114]:
def range_exclude(n, exclude: list):
    # M1: convert to set, then set difference, then convert back to list
    return list(set(range(n)).difference(exclude))
    # return list(set(range(n))- set(exclude))
    
    # M2: slow
    return [i for i in range(n) if i not in exclude]
    
range_exclude(10,[2,3,1,4,9])


[0, 5, 6, 7, 8]

## argmax implementation

In [ ]:
def argmax(t):
    # t can be any sequence
    # all methods except the last can return one max index only
    
    # M0
    # max_index = 0
    # for i in range(1, len(t)):
    #     if t[i] > t[max_index]:
    #         max_index = i
    # return max_index

    # M1: use index method. Works only if t is list or tuple
    # return t.index(max(t))

    # M2: Very good technique!
    # return max((x, i) for i, x in enumerate(t))[1]

    # M3: use the `key` parameter of `max`. very good
    # return max(range(len(t)), key=lambda i: t[i])
    
    # M4: return all max indices
    max_val = max(t)
    return [i for i in range(len(t)) if t[i] == max_val]

t=[2,4,5,7,1,2,3,6]
t=[1,2,1,2,2]
argmax(t)

[1, 3, 4]

## find max and argmax at the same time
if just want to find a single pair, then the fastest is to first find the max index, then retrieve the max val

In [ ]:
def max_and_arg(t):
    # M0:
    # return (max_val, [index1, index2,...])
    # max_val = max(t)
    # return (max_val, [i for i in range(len(t)) if t[i] == max_val])

    # M1
    # return a tuple (max, argmax) of t
    # M1: argmax is the maximum index
    # return max( (x,i)  for i,x in enumerate(t) ) # x is the value, i is the index!

    # M2: use index method of tuple
    # max_=max(t)
    # return max_, t.index(max_)

    # M3: use np.argmax
    max_index=np.argmax(t)
    return t[max_index], max_index
    


t=[1,2,1,2,2]
max_and_arg(t)


(2, 1)

## loop over a range that is the union of two ranges
- EX: (1,10,5,20) => range(1,20)
- the two ranges may have overlaps
- we need to sort the final output

In [190]:
def two_range(i,j,k,w):
    # return a list that contains the union of all the numbers in range(i,j) and range(k,w).
    # M0: wrong method, because the two ranges may have overlaps: e.g., (1,10,5,8)
    # return sorted([i for i in [*range(i,j), *range(k,w)]])

    # M1: 
    # correct, but slow
    # return sorted([i for i in {*range(i,j), *range(k,w)}])

    # M2:
    # form a set first, then sort it 
    # return sorted({*range(i,j), *range(k,w)})
    
    # M3:
    return sorted(set(range(i,j)).union(range(k,w)))

two_range(4,10,22,26) # [4, 5, 6, 7, 8, 9, 22, 23, 24, 25]
two_range(4,10,5,9) # [4, 5, 6, 7, 8, 9]
two_range(1,10,-2,11) # [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

[4, 5, 6, 7, 8, 9, 22, 23, 24, 25]

[4, 5, 6, 7, 8, 9]

[-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## sort two given lists, according to the first
- useful when doing line plots, where we need to sort both x and y arrays, according to x

In [140]:
np.random.seed(1)
x = np.random.normal(size=(100))
y = x + 1+np.random.normal(size=(100))/10
res = sorted(zip(x,y))

x, y = zip(*res)
x[:2]
y[:2]

(-2.3015386968802827, -2.060140709497654)

(-1.2846004535744158, -1.0286247702934248)

## convert list of lists into single list

In [101]:
def to_single_list(t):
    # M1: use itertools.chain.from_iterable
    # return list(itertools.chain.from_iterable(t)) 
    
    # M2: 
    return [x for lis in t for x in lis]

    # M3: use list.extend or `+`
    # res = []
    # for lis in t:
    #     # res.extend(lis)
    #     res += lis
    # return res

t = [[1, 2], [3, 4], [5, 6]] 
to_single_list(t)

[1, 2, 3, 4, 5, 6]

## arithmetic series: sum to a particular number

In [ ]:
def sum_to_end(start=2, end=1000, step=3):
    """calculate the sum from `start` to `end`, with step size = `step`

    Args:
        start (int, optional):  Defaults to 2.
        end (int, optional):  Defaults to 1000.
        step (int, optional):  Defaults to 3.

    Returns:
        int: the required sum
    """    
    # Method 1: C++-like solution using for loop
    ans=0
    for i in range(start,end,step):
        ans+=i
    return ans

    # Method 2: pythonic solution
    return sum(range(2, 1000, 3))

    # Method 3: by the formula for sum of arthemetic series. Fastest.
    n_terms=(end-start)//step + 1
    return n_terms/2 * (2*start+step*(n_terms-1))

# test code
sum_to_end() 

## build frequency distribution

In [3]:
def freq_dist(s):
    # input `s` can be any sequence, not only string!

    # M0. Naive
    # res={}
    # for c in s: 
    #     if c in res:
    #         res[c]+=1
    #     else:
    #         res[c]=1
    # return res

    # M2: use dict.get or dict.setdefault
    res={}
    for c in s:
        res[c]=res.get(c,0)+1
        # res[c]=res.setdefault(c,0)+1
    return res

    # M3: use Counter
    # from collections import Counter 
    # return Counter(s)

    # M4: comprehension, and set + {str, list}.count. Set is used to make code faster 
    # NOTE: it requires the data structure has implemented `.count` method!
    # return {c: s.count(c) for c in set(s)}

def print_freq_dist(d):
    for key, val in sorted(d.items()):
        print(f'{key}: {val}')

    # alternatively
    # for key in sorted(d):
    #     print(f'{key}: {d[key]}')
    
    

s='defvc'
freq_dist(s)
print_freq_dist(freq_dist(s))
t=[1,2,2,1,2,1,'b',"abc"]
freq_dist(t)
        

c: 1
d: 1
e: 1
f: 1
v: 1


{1: 3, 2: 3, 'abc': 1, 'b': 1}

## given a dict, find the key that corresponds to the largest value 
- if value is frequency, then find the one with max frequency
- better to convert the dict to Counter, then use `.most_common(1)[0]`

In [37]:
def largest_value(d):
    # return the entire (key,value) pair
    # return max(d.items(), key=lambda t: t[1])
    
    # return the key only
    return max(d.keys(), key=lambda x: d[x])
    # return max(d.items(), key=lambda t: t[1])[0]
    
    # use Counter
    return Counter(d).most_common(1)[0]

d={0: 4, 1: 3, 2: 3, -1: 2}
largest_value(d)

(0, 4)

## given a dict, reverse the mapping
- `d = {1:2, 2:2, 3:4} `
- `output = {2:[1,2], 4:3}`

In [140]:
def reverse_dict(d):
    # res = dict()
    # for key, val in d.items():
    #     res[val] = res.get(val, []) + [key] 
    # return res

    # M2: can give single value as a scalar, not list
    res = dict()
    for key, val in d.items():
        if val in res:
            if isinstance(res[val], int):
                # print("helo")
                res[val] = [res[val], key]
            else:
                res[val].append(key)
        else:
            res[val] = key
    return res

d = {1:2, 2:2, 3:4}
reverse_dict(d)

{2: [1, 2], 4: 3}

## reverse lookup: given a dict and a value, find all the keys that map to it
- originally, given a key, find its value
- now, given a value, find the keys

In [144]:
def reverse_lookup(d, lookup_val):
    return [key for key, val in d.items() if val == lookup_val]
    # res = []
    # for key, val in d.items():
    #     if val == lookup_val:
    #         res.append(key)
    # return res 

d=dict(a=1,b=2,c=3,d=3)
print(reverse_lookup(d,3)) # ['c', 'd']
# print(reverse_lookup(d,4, default=-2)) # -2
print(reverse_lookup(d,4)) # -2

['c', 'd']
[]


## find most frequent value in a list

In [163]:
def most_frequent(t):
    # M0: use t.count (https://bit.ly/2QHgL7q)
    # in 0th iteration: t[0] will be pass to t.count => find t.count(t[0])
    # return max(t, key = t.count)

    # M1: M0, with set() to improve efficiencys
    # return max(set(t), key = t.count)

    # M2: Counter
    # freq_count=Counter(t)
    # return max(freq_count, key= lambda key: freq_count[key])

    # M3: use the most_common method of Counter
    # return Counter(t).most_common(1)[0][0]

    # M3.5: return all most frequent values. First find the maximum occurrence
    freq_count=Counter(t)
    max_val =  max(freq_count.values())
    return [key for key, val in freq_count.items() if val == max_val]
    # res=[]
    # for key in freq_count.keys():
    #     if freq_count[key] == max(freq_count.values()):
    #         res.append(key)
    # return res

    # M4: use np.unique
    val, count = np.unique(t, return_counts=True)
    return val[count==np.max(count)] # return all most frequent

    return val[np.argmax(count)] # return only one of the most frequent

    # Naive: build a list first
    res={}
    for x in set(t):
        res[x]=t.count(x)
    print(res)
    return max(res.keys(), key=lambda x: res[x])

t=[1,1,1,2,2,2,-1,-1,0]
most_frequent(t)

[1, 2]

## palindrome
- a word, phrase, or sequence that reads the same backwards as forwards, e.g. madam or nurses run
- stack method: the middle+1 element should be the same as middle-1 elements

In [166]:
def is_palindrome(x):
    # slow
    # return list(reversed(x)) == list(x)
    
    return all(x==y for x,y in zip(reversed(x), x))

    return x[::-1]



[is_palindrome(x) for x in ['refer', '112', [1,3,1], np.arange(4)]]

[True, False, True, False]

In [ ]:
def is_palindrome(x):
    # x is a sequence, not necessarily a string!
    # M1: Most efficient method. Also works only for sequence, since reversed requires a sequence
    return all(x == y for x, y in zip(x, reversed(x)))
    # return not any(x!=y for x,y in zip(seq, reversed(seq))) # another way; derived by logic rules
    
    # M2:
    return list(x) == list(reversed(x))

    # M3: Less efficient, because we need to generate the whole t[::-1] first. Also, it works only when t is sequence. In particular, this method does not work for ndarray, since == is meant to be element-wise comparison.
    return x == x[::-1]

    # M4: use a stack. This method is needed when we find the longest palindrome of a string!
    # e.g., s = 'abba'
    # mid = 2, stack = [ab]
    # c should go from 2 til end
    # x = 'abc' => stack =[a]
    # x = 'abcd' => stack =[ab]
    mid = (n:=len(s))//2
    stack = list(s[:mid])
    # if n is odd, we should ignore the middle char!
    for c in s[mid if n%2==0 else mid+1:]: 
        if stack.pop() != c:
            return False
    return True

    # M5: (bad)
    # for i in range(len(seq)):
    #     if(seq[i]!=seq[-i-1]): return False
    # return True
    
    # wrong method
    # return seq==reversed(seq) # we cannot do the following, because comparison must be of the same type. If not, then the comparison must be false!!!

[is_palindrome(x) for x in ['refer', '112', [1,3,1], np.arange(4)]]

[True, False, True, False]

In [ ]:
def is_palindrome_num(x):
    # non-recursive solution
    if x<0: return False # by def, negative numbers are not panlindrome
    return reverse_num(x) == x # using a function is better. Otherwise, we need to create a clone of input x, since we would have modified its value!

    # M2
    """ palindrome for integer """
    if x<=9: return True   
    n_digit=n_digit_rec(x)
    
    leftmost_digit=x//(10**(n_digit-1))
    rightmost_digit=x%10
    x_reduced= x - leftmost_digit*10**(n_digit-1)
    x_reduced//=10
    # print(x_reduced, leftmost_digit, rightmost_digit)
    return leftmost_digit==rightmost_digit and palindrome(x_reduced)

is_palindrome_num(122121)
is_palindrome_num(1221)

False

True

## check whether all elements of a sequence are distinct (i.e., unique)

In [ ]:
def all_unique(s):
    # check whether the elements in s (a sequence) are unique
    # M1. Faster
    # return len(set(s)) == len(s)

    # M2. Use sorting as subroutine. O(nlogn). This method is interesting
    temp = sorted(s)
    return all(s[cur] != s[cur+1] for cur in range(len(temp)-1))

all_unique([2,2,1,3])
all_unique([2,1,3])
all_unique([2,1,3,4])
all_unique([2,1,3,4,5])
all_unique([2,1,3,4,5,5])

False

True

True

True

False

## count the number of unique (distinct) elements of an iterable

In [ ]:
def count_unique(t):
    # # M1: always works, even when t contains unhashable elements
    # t_unique = []
    # for x in t:
    #     if x not in t_unique:
    #         t_unique.append(x)
    # return len(t_unique)

    # # M2: this method doesn't work when t contains unhashable items, such as a list
    # return len(set(t))

    # M3: only works if container contain hashable items only
    return len(Counter(t))
    
    # M4: use np.unique then return the size attribute
    # Works with ndarray, since unique method can be used when ndim>1. Doesn't work with string
    # return np.unique(t).size
    
t=[1,2,3,4,3,3,3]
# t2=[1,2,3,4,[1,2,3]]
s="AAaaBb"
a_3d=np.random.randint(20,size=(3,4,5)) 

count_unique(t)
# count_unique(t2)
count_unique(s)


4

4

# combination problems

## find r-combinations

In [62]:
def comb_2(t):
    res = [(t[i],t[j]) for i in range(len(t)) for j in range(i+1, len(t))]
    print(f"{len(res) = }")
    return res 

comb_2([1,2,3])

len(res) = 3


[(1, 2), (1, 3), (2, 3)]

In [81]:
def combs(t, r):
    # M1: use combinations function in itertools
    from itertools import combinations 
    return list(combinations(t,r)) # the function returns an iterator. If returning a list is required, use list()
    
    # M2: use Pascal identity. That is, the last element is either chosen or not chosen. 
    # choose last element or not?
    # nCr = (n-1)C(r-1) + (n-1)Cr
    return combs_aux(t,len(t), r)

def combs_aux(t, n, r):
    if r ==0: return [[]]
    if r==n: return [t[:n]]
    return sorted([s + [t[n-1]] for s in combs_aux(t, n-1, r-1) ] + combs_aux(t, n-1, r))
s=[1,2,3,4,5]
print(combs(s,3))
# [(1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]

5 2
4 1
3 0
3 1
2 0
2 1
1 0
1 1
4 2
3 1
2 0
2 1
1 0
1 1
3 2
2 1
1 0
1 1
2 2


[[1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [2, 3],
 [2, 4],
 [2, 5],
 [3, 4],
 [3, 5],
 [4, 5]]

## find powerset

# string manipulation
- remember that to change to upper/lower letter for a given x, first express x in terms of 'a'. I.e., x = 'a'+ ?, for some ?. Then X = 'A'+?  (X represents the upper letter). So, 
X='A'+x-'a'
- `str.join` is to convert the given sequence to a list

In [9]:
s = 'ERIWOFJIOREJklklm fekfkem e#$%^'
s.swapcase()
''.join(
    chr(ord(c) - ord('a') + ord('A')) if c.islower() else chr(ord(c) - ord('A') + ord('a')) if c.isupper() else c for c in s
    )
        

'eriwofjiorejKLKLM FEKFKEM E#$%^'

## make a dict with {1:'a', ..., 26:'z'}

In [15]:
print(dict(zip(range(1,27), ''.join(chr(ord('a') + i) for i in range(26)))))

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


## swap cases for alphabets

In [12]:
s = 'ERIWOFJIOREJklklm fekfkem e#$%^'

# M1: use existing method
s.swapcase()

# M2: self-defined function
# alternatively, use `s.upper()` and `s.lower()`
def swapcase_(c):
    low_to_up = - ord('a') + ord('A')
    return (
        chr(ord(c) + low_to_up) if c.islower() 
        else chr(ord(c) - low_to_up) if c.isupper()
        else c
        )
    # return (
    #     chr(ord(c) - ord('a') + ord('A')) if 'a'<=c<='z' 
    #     else chr(ord(c) - ord('A') + ord('a')) if 'A'<=c<='Z' 
    #     else c
    #     )
''.join(swapcase_(c) for c in s)

'eriwofjiorejKLKLM FEKFKEM E#$%^'

## count how many lower and upper english letters

In [90]:
def string_summary(s):
    # M1: use a list such that res[0] = # low letter, res[1] = # up letter, res[2] = others
    # res = [0,0,0]
    # for c in s:
    #     res[0 if c.islower() else 1 if c.isupper() else 2] +=1
    # return res[0], res[1]

    # M2:
    n_up = n_low = 0
    for c in s:
        if 'a'<=c<='z': # DON'T use ord() to convert to integer!!!!!!
            n_low += 1
        elif 'A'<=c<='Z':
            n_up += 1
    return n_low, n_up

    # best: use dict
    
    # res = dict.fromkeys(['up', 'low', 'non_alpha'], 0)
    # for c in s:
    #     # bad method:
    #     # if c.isupper():
    #     #     res['up'] += 1
    #     # elif c.islower():
    #     #     res['low'] += 1
    #     # else:
    #     #     res['non_alpha'] += 1
        
    #     # good method
    #     res['up' if c.isupper() else 'low' if c.islower() else 'non_alpha'] += 1
    # return res

string_summary('frnekfneEWKKNEWIONFIONKdaa23456789#$%^&*dfghjkcvbnm')

(22, 15)

## use `join` to create a string with all English letters

In [111]:
# low
low = [chr(i) for i in range(ord('a'), ord('z')+1)]

# up
up = [chr(ord('A')+i) for i in range(26)]

# all english letters, up and low
all = [chr(i) for i in list(range(ord('a'), ord('z')+1)) + list(range(ord('A'), ord('Z')+1))]
all = [x for i in range(26) for x in [chr(ord('a')+i), chr(ord('A')+i)]] # good method. This allows us to create two characters in each iteration!

# convert to string
''.join(low)

# generate aAbBcC...
''.join(x+X for x, X in zip(low, up))

# M2: use map 
low_letter=''.join(map(chr, range(ord('a'),ord('z')+1)))
up_letter=''.join(map(chr, range(ord('A'),ord('Z')+1)))
low_letter+up_letter
''.join(map(lambda t: t[0]+t[1],zip(low_letter, up_letter)))


'abcdefghijklmnopqrstuvwxyz'

'aAbBcCdDeEfFgGhHiIjJkKlLmMnNoOpPqQrRsStTuUvVwWxXyYzZ'

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

'aAbBcCdDeEfFgGhHiIjJkKlLmMnNoOpPqQrRsStTuUvVwWxXyYzZ'

## generate alphabet string with offset (i.e., start not from 'a') and output size
- if exceed output size, then print characters circularly!

In [6]:
def gen_alphabet(offset , n_char, upper=False, circular=True):
    # offset = 0 => start from 'a'/'A'
    
    start = 'A' if upper else 'a'
    return ''.join(chr((ord(start) + i%26)) for i in range(offset, offset+n_char))

gen_alphabet(2, 10)  # 'cdefghijkl'
gen_alphabet(10, 30)  # 'cdefghijkl'
gen_alphabet(10, 30, upper=True)  # 'cdefghijkl'
        

'cdefghijkl'

'klmnopqrstuvwxyzabcdefghijklmn'

'KLMNOPQRSTUVWXYZABCDEFGHIJKLMN'

## anagram
- Two strings are anagram if they contain the same characters but the order of the characters may be different. For example, 'abcde' and 'deabc' are anagram. 
- remember in the anagram problem, we don't need to create two frequency distribution! Just deduct frequency of 2nd one from the 1st one!!

In [7]:
def freq_dist(s):
    res={}
    for c in s:
        res[c]=res.get(c, 0)+1
    return res

def is_anagram(a,b):
    # check if strings a and b are anagram

    # wrong method: change them to sets, then check whether the two sets are the same. Recall that set eliminates duplicates. if a='aaa', b='aa', it will still return True..
    # return set(a)==set(b)

    # M1: easiest: use Counter
    # return Counter(a)==Counter(b)

    # M2: build freq distribution ourselves
    # return freq_dist(a)==freq_dist(b)

    # M3: sorting first
    # return sorted(a)==sorted(b)
    
    # M4: optimization: build one freq distribution only. Then decrease the count for each char in another string, and check if char has frequency 0
    # freq=freq_dist(a)
    freq=Counter(a)
    for c in b:
        freq[c]=freq.get(c,0)-1
    print(freq)
    return all(val==0 for val in freq.values())

    # M5: M4, but with further optimization
    # freq=freq_dist(a)
    # for c in b:
    #     freq[c]=freq.get(c,0)-1
    #     if freq[c]<0: return False

    # we've ensured that no entries with value<0
    # for c in a:
    #     if freq[c]!=0: return False
    # return True
    
a='bab'
b='bab'
is_anagram(a,b)
is_anagram('abcde', 'deabc')
is_anagram('abce', 'deabc')
is_anagram('deabc', 'abce')

Counter({'b': 0, 'a': 0})


True

Counter({'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0})


True

Counter({'a': 0, 'b': 0, 'c': 0, 'e': 0, 'd': -1})


False

Counter({'d': 1, 'e': 0, 'a': 0, 'b': 0, 'c': 0})


False

## check if a string contains all the characters of another test string
- basically, check whether a set is a subset of another

In [99]:
def contain(s, test):
    # M1
    # for c in test:
    #     if c not in s:
    #         return False
    # return True 
    
    # M2
    return (set(s) >= set(test))


print(contain('abc','ab')) # True
print(contain("abc",'a')) # True
print(contain("abc",'bc')) # True
print(contain("abc",'abc')) # True
print(contain("abc",'axc')) # False
print(contain("abc",'cab')) # True
print(contain("abc",'cabbbbb')) # True

True
True
True
True
False
True
True


In [173]:
def contain(s, test):
    # check if s >= test
    # check if s contains all the characters of `test`
        # if so, then `test` is a subset of `s`
    # check if c in s, for all c in test
    
    # M1: use `>=`
    # return set(s) >= set(test)
    
    # M2: use `issubset`
    return set(test).issubset(s)

    # M3: naive
    # for c in test:
    #     if c not in s:
    #         return False
    # return True 

print(contain('abc','ab')) # True
print(contain("abc",'a')) # True
print(contain("abc",'bc')) # True
print(contain("abc",'abc')) # True
print(contain("abc",'axc')) # False
print(contain("abc",'cab')) # True
print(contain("abc",'cabbbbb')) # True

True
True
True
True
False
True
True
